# Project: Netflix Recommendation System „Movie’s platform”


# 0. Import necessary libraries

In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

# Import pandas and numpy for data analysis
import pandas as pd
import numpy as np

# To display all rows
pd.set_option('display.max_rows', 100000)

# To display all columns
pd.set_option('display.max_columns', 100000)

# For date functions
from datetime import datetime as dt

# For scraping web to get countries with their respective languages
import bs4 as bs
import urllib.request

# For creating a connection engine to MySQL and pushing the final cleaned data into the DB
from sqlalchemy import create_engine
import pymysql


# For pushing data into MySQL, we need connection string to MySQL.
# Connection details are stored in .env file
# Hence, loading dotenv library to access .env file
%load_ext dotenv
%dotenv mysql_connection.env
import os

# 1. Data Reading and Understanding

In [2]:
# Define a function that returns the first 5 records, null percentage of each column and 
# statistical information of the dataset passed in parameter
def data_understanding (df):
  return df.head(), round(100*(df.isnull().sum()/len(df.index)), 2), df.describe();

### 1.1. netflix_titles Dataset

In [3]:
# Import the dataset and call the data understanding function
netflix_titles = pd.read_csv('..\\Datasets\\Netflix\\netflix_titles.csv')
head, null_perc, stats = data_understanding(netflix_titles)

In [4]:
# Display the first 5 entries
display(head)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


#### Observations:

> 1.   The actual name of the show is not given. Instead they are identified by show_id.
> 2.   Shows can be series/movie. We need to explore if there are other categories of shows available in this dataset.
> 3. The column 'title' is ambiguous. It has percentage, something in duration or just plain numbers. We need to clarify this.
> 4. There are some null values present. We need to clean them.

In [5]:
# Get more info on the dataset like datatypes, number of records, etc.
netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
show_id         7787 non-null object
type            7787 non-null object
title           7787 non-null object
director        5398 non-null object
cast            7069 non-null object
country         7280 non-null object
date_added      7777 non-null object
release_year    7787 non-null int64
rating          7780 non-null object
duration        7787 non-null object
listed_in       7787 non-null object
description     7787 non-null object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [6]:
# Get percentage of null values for each column
display(null_perc)

show_id          0.00
type             0.00
title            0.00
director        30.68
cast             9.22
country          6.51
date_added       0.13
release_year     0.00
rating           0.09
duration         0.00
listed_in        0.00
description      0.00
dtype: float64

#### Observations
> 1.   There are about 7.7k records
> 2.   Director, cast and country have some null values, which need to be handled later.
> 3. Date columns have to be converted to datetime type or can be extracted to new features like weekdays/weekends, month and year.

In [7]:
# Get statistical information on the release_year (the only integer column) like percentile, min, max, etc.
display(stats)

,release_year
count,7787.000000
mean,2013.932580
std,8.757395
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2018.000000
max,2021.000000


#### Observation:
> 1. The oldest show on netflix was released in 1925 and the latest was released in 2021. The dataset is pretty much updated.
> 2. It can be noticed that not many movies are in Netflix that were released between the years 1925 and 2013. They all lie in the first 25th percentile.

### 1.2. Prime TV Shows Dataset

In [8]:
# Import the dataset and call the data understanding function
prime_shows = pd.read_csv('..\\Datasets\\Amazon_Prime_Video_Shows\\Prime TV Shows Data set.csv', encoding = "ISO-8859-1") 
head, null_perc, stats = data_understanding(prime_shows)

In [9]:
# Display the first 5 records
display(head)

,S.no.,Name of the show,Year of release,No of seasons available,Language,Genre,IMDb rating,Age of viewers
0,1,Pataal Lok,2020.0,1.0,Hindi,Drama,7.5,18+
1,2,Upload,2020.0,1.0,English,Sci-fi comedy,8.1,16+
2,3,The Marvelous Mrs. Maisel,2017.0,3.0,English,"Drama, Comedy",8.7,16+
3,4,Four More Shots Please,2019.0,2.0,Hindi,"Drama, Comedy",5.3,18+
4,5,Fleabag,2016.0,2.0,English,Comedy,8.7,18+


#### Observations:

> 1.   Here, the name of the show is given unlike the previous data set.
> 2.   The features are self-explanatory.
> 3. The column Year of release has years in decimals.

In [10]:
# Get more info on the dataset like datatypes, number of records, etc.
prime_shows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 8 columns):
S.no.                      404 non-null int64
Name of the show           393 non-null object
Year of release            393 non-null float64
No of seasons available    393 non-null float64
Language                   393 non-null object
Genre                      393 non-null object
IMDb rating                182 non-null float64
Age of viewers             393 non-null object
dtypes: float64(3), int64(1), object(4)
memory usage: 25.4+ KB


In [11]:
# Get percentage of null values for each column
display(null_perc)

S.no.                       0.00
Name of the show            2.72
Year of release             2.72
No of seasons available     2.72
Language                    2.72
Genre                       2.72
IMDb rating                54.95
Age of viewers              2.72
dtype: float64

#### Observations:
> 1. It is a very small dataset with just 404 records.
> 2. It cab be seen that only S.no has 404 records while most of the columns have only 393. This means that the actual number of records that will be useful is 393. We need to remove the other entries.
> 3. IMDb rating has only 182 entries that are not null, which is less than half of the total records. 

In [12]:
# Get statistical information like percentile, min, max, etc.
display(stats)

,S.no.,Year of release,No of seasons available,IMDb rating
count,404.000000,393.000000,393.000000,182.000000
mean,202.500000,2011.279898,2.608142,7.354396
std,116.769003,12.944861,2.592008,0.959372
min,1.000000,1926.000000,1.000000,3.700000
25%,101.750000,2011.000000,1.000000,6.900000
50%,202.500000,2016.000000,2.000000,7.500000
75%,303.250000,2018.000000,3.000000,8.100000
max,404.000000,2020.000000,20.000000,9.000000


#### Observations:
> 1. The oldest TV show present was released in 1926 and the latest in 2020.
> 2. Most of the shows present were released between 2016 and 2020.
> 3. The maximum number of seasons available is 20, which can be an outlier.

### 1.3. Rotten Tomatoes Dataset

#### 1.3.1. Rotten tomatoes movies dataset


In [13]:
# Import the dataset and call the data understanding function
rt_movies = pd.read_csv('..\\Datasets\\Rotten_Tomatoes\\rotten_tomatoes_movies.csv') 
head, null_perc, stats = data_understanding(rt_movies)

In [14]:
# Display the first 5 records
display(head)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,2012-09-04,90.0,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,2014-07-24,122.0,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,2017-01-13,95.0,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,2016-06-10,127.0,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


#### Observations:
> The dataset gives a complete idea about a movie's ratings, genre, runtime and even audience count, top-critics count, etc.

In [15]:
# Get more info on the dataset like datatypes, number of records, etc.
rt_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17712 entries, 0 to 17711
Data columns (total 22 columns):
rotten_tomatoes_link                17712 non-null object
movie_title                         17712 non-null object
movie_info                          17391 non-null object
critics_consensus                   9134 non-null object
content_rating                      17712 non-null object
genres                              17693 non-null object
directors                           17518 non-null object
authors                             16170 non-null object
actors                              17360 non-null object
original_release_date               16546 non-null object
streaming_release_date              17328 non-null object
runtime                             17398 non-null float64
production_company                  17213 non-null object
tomatometer_status                  17668 non-null object
tomatometer_rating                  17668 non-null float64
tomatometer_count   

In [16]:
# Display null percentage of each column
display(null_perc)

rotten_tomatoes_link                 0.00
movie_title                          0.00
movie_info                           1.81
critics_consensus                   48.43
content_rating                       0.00
genres                               0.11
directors                            1.10
authors                              8.71
actors                               1.99
original_release_date                6.58
streaming_release_date               2.17
runtime                              1.77
production_company                   2.82
tomatometer_status                   0.25
tomatometer_rating                   0.25
tomatometer_count                    0.25
audience_status                      2.53
audience_rating                      1.67
audience_count                       1.68
tomatometer_top_critics_count        0.00
tomatometer_fresh_critics_count      0.00
tomatometer_rotten_critics_count     0.00
dtype: float64

#### Observations:
> 1. Dataset has 17k entires with 22 columns.
> 2. There are a few columns with less than 10% of null values, while the column Critics Consensus has 48% of null values which need to be handled.
> 3. The dates are in object format which needs to be converted to datetime to get a better understanding.

In [17]:
# Get statistical information like percentile, min, max, etc.
display(stats)

,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,17398.000000,17668.000000,17668.000000,17416.000000,1.741500e+04,17712.000000,17712.000000,17712.000000
mean,102.214048,60.884763,57.139801,60.554260,1.439401e+05,14.586326,36.374831,20.703139
std,18.702511,28.443348,68.370047,20.543369,1.763577e+06,15.146349,52.601038,30.248435
min,5.000000,0.000000,5.000000,0.000000,5.000000e+00,0.000000,0.000000,0.000000
25%,90.000000,38.000000,12.000000,45.000000,7.075000e+02,3.000000,6.000000,3.000000
50%,99.000000,67.000000,28.000000,63.000000,4.277000e+03,8.000000,16.000000,8.000000
75%,111.000000,86.000000,75.000000,78.000000,2.498800e+04,23.000000,44.000000,24.000000
max,266.000000,100.000000,574.000000,100.000000,3.579764e+07,69.000000,497.000000,303.000000


#### Observations:
> 1. There are a few movies which don't have ratings at all.
> 2. The ratings count for a few movies are huge. Those movies might quite popular.

#### 1.3.2. Rotten tomatoes critics dataset

In [18]:
# Import the dataset and call the data understanding function
rt_critics = pd.read_csv('..\\Datasets\\Rotten_Tomatoes\\rotten_tomatoes_critic_reviews.csv') 
head, null_perc, stats = data_understanding(rt_critics)

In [19]:
# Display the first 5 records
display(head)

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


#### Observations:
> 1. Both the rotten tomatoes datasets have common column - rotten_tomatoes_link. These two datasets can be joined to get a bigger picture.
> 2. We can use review content and get some key words to understand better about the movie's performance.

In [20]:
# Get more info on the dataset like datatypes, number of records, etc.
rt_critics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130017 entries, 0 to 1130016
Data columns (total 8 columns):
rotten_tomatoes_link    1130017 non-null object
critic_name             1111488 non-null object
top_critic              1130017 non-null bool
publisher_name          1130017 non-null object
review_type             1130017 non-null object
review_score            824081 non-null object
review_date             1130017 non-null object
review_content          1064211 non-null object
dtypes: bool(1), object(7)
memory usage: 61.4+ MB


In [21]:
# Display null percentage of each column
display(null_perc)

rotten_tomatoes_link     0.00
critic_name              1.64
top_critic               0.00
publisher_name           0.00
review_type              0.00
review_score            27.07
review_date              0.00
review_content           5.82
dtype: float64

#### Observations:
> 1. There are 1.1 Million entries in the dataset with 8 features.
> 2. Review score has 27% null values, which we need to handle.
> 3. Almost all the columns are of object datatype. And there is one column of boolean datatype.

### 1.4. IMDb Dataset

#### 1.4.1. IMDb Movies Dataset

In [22]:
# Import the dataset and call the data understanding function
imdb_movies = pd.read_csv('..\\Datasets\\iMDB\\IMDb movies.csv') 
head, null_perc, stats = data_understanding(imdb_movies)

In [23]:
# Display the first 5 records
display(head)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


#### Observations:
> 1. The dataset is similar to netflix dataset with some additional information like votes, user reviews, etc.
> 2. There might be a lot of null values. We will find out about that in the next.

In [24]:
# Get more info on the dataset like datatypes, number of records, etc.
imdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 22 columns):
imdb_title_id            85855 non-null object
title                    85855 non-null object
original_title           85855 non-null object
year                     85855 non-null object
date_published           85855 non-null object
genre                    85855 non-null object
duration                 85855 non-null int64
country                  85791 non-null object
language                 85022 non-null object
director                 85768 non-null object
writer                   84283 non-null object
production_company       81400 non-null object
actors                   85786 non-null object
description              83740 non-null object
avg_vote                 85855 non-null float64
votes                    85855 non-null int64
budget                   23710 non-null object
usa_gross_income         15326 non-null object
worlwide_gross_income    31016 non-null object

In [25]:
# Display null percentage of each column
display(null_perc)

imdb_title_id             0.00
title                     0.00
original_title            0.00
year                      0.00
date_published            0.00
genre                     0.00
duration                  0.00
country                   0.07
language                  0.97
director                  0.10
writer                    1.83
production_company        5.19
actors                    0.08
description               2.46
avg_vote                  0.00
votes                     0.00
budget                   72.38
usa_gross_income         82.15
worlwide_gross_income    63.87
metascore                84.50
reviews_from_users        8.85
reviews_from_critics     13.74
dtype: float64

#### Observations:
> 1. We have 85k entries with 22 columns.
> 2. Few columns like description, production_company, etc. have less percentage of null values.
> 3. Few columns like budget, gross incomes, etc have more than 60%-80% of null values.

In [26]:
# Get statistical information like percentile, min, max, etc.
display(stats)

,duration,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics
count,85855.000000,85855.000000,8.585500e+04,13305.000000,78258.000000,74058.000000
mean,100.351418,5.898656,9.493490e+03,55.896881,46.040826,27.479989
std,22.553848,1.234987,5.357436e+04,17.784874,178.511411,58.339158
min,41.000000,1.000000,9.900000e+01,1.000000,1.000000,1.000000
25%,88.000000,5.200000,2.050000e+02,43.000000,4.000000,3.000000
50%,96.000000,6.100000,4.840000e+02,57.000000,9.000000,8.000000
75%,108.000000,6.800000,1.766500e+03,69.000000,27.000000,23.000000
max,808.000000,9.900000,2.278845e+06,100.000000,10472.000000,999.000000


#### Observations:
Most of the features seem to be uniformly distributed. There are some outliers in the reviews columns.

#### 1.4.2. IMDb Names Dataset

In [27]:
# Import the dataset and call the data understanding function
imdb_names = pd.read_csv('..\\Datasets\\iMDB\\IMDb names.csv') 
head, null_perc, stats = data_understanding(imdb_names)

In [28]:
# Display the first 5 records
display(head)

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death,spouses_string,spouses,divorces,spouses_with_children,children
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987-06-22,"Los Angeles, California, USA",pneumonia,Robyn Smith (27 June 1980 - 22 June 1987) (hi...,2,0,1,2
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014-08-12,"New York City, New York, USA",stroke,Jason Robards (4 July 1961 - 10 September 196...,2,1,2,3
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,Bernard d'Ormale (16 August 1992 - present)\n...,4,3,1,1
3,nm0000004,John Belushi,John Adam Belushi,170.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,Judith Belushi-Pisano (31 December 1976 - 5 M...,1,0,0,0
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden (...",2007-07-30,"Fårö, Gotlands län, Sweden",natural causes,Ingrid Bergman (11 November 1971 - 20 May 199...,5,4,5,8


#### Observations:
> We can see that this dataset is a collection of information about people accociated with the industry like their bio, DOB, DOD, spouses, children, etc

In [29]:
# Get more info on the dataset like datatypes, number of records, etc.
imdb_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297705 entries, 0 to 297704
Data columns (total 17 columns):
imdb_name_id             297705 non-null object
name                     297705 non-null object
birth_name               297705 non-null object
height                   44681 non-null float64
bio                      204698 non-null object
birth_details            110612 non-null object
date_of_birth            110612 non-null object
place_of_birth           103992 non-null object
death_details            39933 non-null object
date_of_death            39933 non-null object
place_of_death           37038 non-null object
reason_of_death          22694 non-null object
spouses_string           45352 non-null object
spouses                  297705 non-null int64
divorces                 297705 non-null int64
spouses_with_children    297705 non-null int64
children                 297705 non-null int64
dtypes: float64(1), int64(4), object(12)
memory usage: 38.6+ MB


In [30]:
# Display null percentage of each column
display(null_perc)

imdb_name_id              0.00
name                      0.00
birth_name                0.00
height                   84.99
bio                      31.24
birth_details            62.85
date_of_birth            62.85
place_of_birth           65.07
death_details            86.59
date_of_death            86.59
place_of_death           87.56
reason_of_death          92.38
spouses_string           84.77
spouses                   0.00
divorces                  0.00
spouses_with_children     0.00
children                  0.00
dtype: float64

#### Observations:
> The dataset has about 297k records, out of which most of the columns have more than 60% null values. Let us explore the other dataset and find out if we can extract some useful information from this one.

#### 1.4.3. IMDb Ratings Dataset

In [31]:
# Import the dataset and call the data understanding function
imdb_ratings = pd.read_csv('..\\Datasets\\iMDB\\IMDb ratings.csv') 
head, null_perc, stats = data_understanding(imdb_ratings)

In [32]:
# Display the first 5 records
display(head)

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
1,tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,103,28,20,13,16,6.0,1.0,6.1,114.0,6.0,239.0,6.3,115.0,6.1,425.0,6.0,1.0,6.2,102.0,6.0,210.0,6.2,100.0,6.2,50.0,NaN,NaN,5.9,12.0,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
2,tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,32,16,5,6,4,NaN,NaN,5.5,25.0,5.8,72.0,6.2,62.0,5.9,146.0,NaN,NaN,5.5,21.0,5.9,67.0,6.2,55.0,5.7,15.0,NaN,NaN,5.8,4.0,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
3,tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,117,63,26,25,16,NaN,NaN,5.3,23.0,5.0,111.0,5.3,193.0,5.1,299.0,NaN,NaN,5.2,20.0,4.9,96.0,5.2,171.0,5.9,39.0,NaN,NaN,5.7,3.0,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
4,tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0


#### Observations:
> 1. This dataset gives us more details about the ratings of the movies. This data can be combined with the movies dataset to get complete information.
> 2. There are a few columns which have ambiguous names. We need to find out what exactly the column represents.

In [33]:
# Get more info on the dataset like datatypes, number of records, etc.
imdb_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 49 columns):
imdb_title_id                85855 non-null object
weighted_average_vote        85855 non-null float64
total_votes                  85855 non-null int64
mean_vote                    85855 non-null float64
median_vote                  85855 non-null float64
votes_10                     85855 non-null int64
votes_9                      85855 non-null int64
votes_8                      85855 non-null int64
votes_7                      85855 non-null int64
votes_6                      85855 non-null int64
votes_5                      85855 non-null int64
votes_4                      85855 non-null int64
votes_3                      85855 non-null int64
votes_2                      85855 non-null int64
votes_1                      85855 non-null int64
allgenders_0age_avg_vote     33359 non-null float64
allgenders_0age_votes        33359 non-null float64
allgenders_18age_avg_vote    8

In [34]:
# Display null percentage of each column
display(null_perc)

imdb_title_id                 0.00
weighted_average_vote         0.00
total_votes                   0.00
mean_vote                     0.00
median_vote                   0.00
votes_10                      0.00
votes_9                       0.00
votes_8                       0.00
votes_7                       0.00
votes_6                       0.00
votes_5                       0.00
votes_4                       0.00
votes_3                       0.00
votes_2                       0.00
votes_1                       0.00
allgenders_0age_avg_vote     61.14
allgenders_0age_votes        61.14
allgenders_18age_avg_vote     0.82
allgenders_18age_votes        0.82
allgenders_30age_avg_vote     0.01
allgenders_30age_votes        0.01
allgenders_45age_avg_vote     0.09
allgenders_45age_votes        0.09
males_allages_avg_vote        0.00
males_allages_votes           0.00
males_0age_avg_vote          68.07
males_0age_votes             68.07
males_18age_avg_vote          1.71
males_18age_votes   

#### Observations:
> 1. The dataset has 85k records with a total of 49 columns.
> 2. Few columns have more than 60% null values, while others have a lesser percentage. We can handle them in the data cleaning section.
> 3. We will look into the statistical information next.

In [35]:
# Get statistical information like percentile, min, max, etc.
display(stats)

,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
count,85855.000000,8.585500e+04,85855.000000,85855.000000,8.585500e+04,85855.000000,85855.000000,85855.000000,85855.000000,85855.000000,85855.000000,85855.000000,85855.000000,85855.000000,33359.000000,33359.000000,85149.000000,85149.000000,85845.000000,85845.000000,85775.000000,85775.000000,85854.000000,8.585400e+04,27411.000000,27411.000000,84390.000000,84390.000000,85843.000000,85843.000000,85754.000000,85754.000000,85774.000000,85774.000000,22117.000000,22117.000000,79334.000000,79334.000000,84911.000000,84911.000000,83057.000000,83057.000000,85176.000000,85176.000000,85646.000000,85646.000000,85854.000000,85854.000000
mean,5.898656,9.493490e+03,6.226342,6.267853,1.216158e+03,1196.774608,2040.896337,2105.668371,1353.192802,700.291177,340.179628,192.263409,126.663852,221.401479,6.195135,15.108097,6.011314,1778.370809,5.862917,4053.927124,5.678195,1260.706803,5.817172,6.029016e+03,6.221407,12.436978,5.946389,1343.226970,5.817767,3324.669338,5.630150,1051.196877,6.039408,1366.759461,6.147308,4.842881,6.321509,445.072340,6.071223,685.655451,6.022226,196.065341,5.155706,76.110231,5.925910,1647.822537,5.749638,4332.940865
std,1.234987,5.357436e+04,1.155280,1.481168,1.245582e+04,11735.363316,13801.122865,10699.375191,5976.801303,2931.301270,1415.815848,814.505546,551.690563,1172.132946,1.849155,74.016475,1.439783,11798.156167,1.279443,22087.819201,1.250633,5363.599259,1.248058,3.364652e+04,1.767590,56.672675,1.464165,9119.273345,1.289268,18348.499984,1.264053,4473.652738,1.346245,7672.675428,1.878342,16.045455,1.738922,2830.024831,1.454370,3732.376186,1.541895,842.979585,1.290920,124.264020,1.308087,8455.978158,1.285570,23243.807279
min,1.000000,9.900000e+01,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,5.200000,2.050000e+02,5.600000,6.000000,2.100000e+01,8.000000,18.000000,28.000000,30.000000,22.000000,12.000000,7.000000,5.000000,10.000000,5.000000,1.000000,5.300000,13.000000,5.100000,69.000000,5.000000,53.000000,5.100000,1.320000e+02,5.000000,1.000000,5.100000,9.000000,5.100000,56.000000,4.900000,45.000000,5.300000,22.000000,5.000000,1.000000,5.500000,4.000000,5.300000,10.000000,5.200000,6.000000,4.400000,11.000000,5.200000,24.000000,5.000000,95.000000
50%,6.100000,4.840000e+02,6.400000,6.000000,5.100000e+01,24.000000,49.000000,77.000000,76.000000,53.000000,30.000000,19.000000,14.000000,25.000000,6.000000,2.000000,6.200000,43.000000,6.100000,172.000000,5.900000,123.000000,6.000000,3.080000e+02,6.000000,1.000000,6.200000,31.000000,6.000000,140.000000,5.900000,104.000000,6.300000,58.000000,6.000000,1.000000,6.500000,11.000000,6.300000,27.000000,6.200000,16.000000,5.300000,29.000000,6.100000,81.000000,6.000000,225.000000
75%,6.800000,1.766500e+03,7.000000,7.000000,1.920000e+02,111.500000,234.000000,342.000000,290

#### Observations:
> 1. Clearly we can say that statistical information for all the columns seem to be correct.
> 2. All the scores lie in the range of 1 to 10.
> 3. There are no negative numbers.
> 4. The number of votes are not evenly distributed, but it is not an issue because some movies can be way more popular than the others. Hence the imbalance. 

#### 1.4.4. IMDb Title Principals Dataset

In [36]:
# Import the dataset and call the data understanding function
imdb_title_principals = pd.read_csv('..\\Datasets\\iMDB\\IMDb title_principals.csv') 
head, null_perc, stats = data_understanding(imdb_title_principals)

In [37]:
# Display the first 5 records
display(head)

,imdb_title_id,ordering,imdb_name_id,category,job,characters
0,tt0000009,1,nm0063086,actress,NaN,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,tt0000009,2,nm0183823,actor,NaN,"[""Mr. Hamilton""]"
2,tt0000009,3,nm1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor..."
3,tt0000009,4,nm0085156,director,NaN,NaN
4,tt0000574,1,nm0846887,actress,NaN,"[""Kate Kelly""]"


#### Observations:
> 1. This dataset gives us information about the actor/actress/director, etc. who are accociated with a particular movie listed in IMDb movies. The complete information about the people can be found in the IMDb names dataset. 
> 2. The data also gives us info about the characters played by these people.

In [38]:
# Get more info on the dataset like datatypes, number of records, etc.
imdb_title_principals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835513 entries, 0 to 835512
Data columns (total 6 columns):
imdb_title_id    835513 non-null object
ordering         835513 non-null int64
imdb_name_id     835513 non-null object
category         835513 non-null object
job              212731 non-null object
characters       340836 non-null object
dtypes: int64(1), object(5)
memory usage: 38.2+ MB


In [39]:
# Display null percentage of each column
display(null_perc)

imdb_title_id     0.00
ordering          0.00
imdb_name_id      0.00
category          0.00
job              74.54
characters       59.21
dtype: float64

#### Observations:
> 1. The dataset has around 835k records with 6 columns.
> 2. job and characters have more than 50% null values while other columns don't have any.
> 3. We shall not use the datasets - imdb_title_principles and imdb_names, because they give personal information like marital status, children, etc about actors, directors and other personalities assiciated with the movie. Clearly we do not need this information at the moment because it does not really match with the goals of the project, i.e. to do an analysis on the kind of movies/tv shows that would be profitable for Netflix.

# 2. Data Cleaning and Preparation

## 2.1 Data cleaning and preparation for Netflix Dataset

In [40]:
#We find that some of the columns have no values and are shown as NaN. Replace these values to "Unknown". 
#Deleting these rows will result in loss of petentially valuable data.
netflix_titles["director"] = netflix_titles["director"].fillna("Unknown")
netflix_titles["cast"]     = netflix_titles["cast"].fillna("Unknown")
netflix_titles["country"]  = netflix_titles["country"].fillna("Unknown")

In [41]:
#Convert the date_added column from string to proper date-time format
#Remove the ',' character in the date and strip the leading and trailing spaces
netflix_titles["date_added"]=netflix_titles["date_added"].str.replace(",", "").str.strip()
#Converting date column from string format to date-time format
netflix_titles["date_added"]=pd.to_datetime(netflix_titles["date_added"], format="%B %d %Y")

In [42]:
#Add new column to capture the duration of TV shows which is in seasons
netflix_titles["duration_season"]=""

#Reorder the columns
column_name = ["show_id", "type", "title", "director", "cast", "country", "date_added", "release_year", "rating",
                "duration", "duration_season", "listed_in", "description"]
netflix_titles = netflix_titles.reindex(columns=column_name)

#Fill the duration_season column with the season data available in the duration column
netflix_titles["duration_season"] = netflix_titles[netflix_titles["duration"].str.contains("Season")]["duration"]

#Clean the duration_season column and make it to integer type
netflix_titles["duration_season"] = netflix_titles["duration_season"].fillna("0")
netflix_titles["duration_season"] = netflix_titles["duration_season"].str.replace("Season", "").str.replace("s", "")
netflix_titles["duration_season"] = netflix_titles["duration_season"].astype(str).astype(int)

In [43]:
#Remove the season data and convert the duration column into integer type
#Remove "Season" information from the column
netflix_titles["duration"]=netflix_titles.duration.str.replace('^(\d+)(.Seasons*)$', "0") 
#Remove "min" information from the column
netflix_titles["duration"]=netflix_titles["duration"].str.replace(" min", "") 
netflix_titles["duration"]=netflix_titles["duration"].astype(int)

#rename the duration column
netflix_titles.rename(columns = {'duration':'duration_min'}, inplace = True)

In [44]:
#Function to check missing values
def missing_values(n):
    df=pd.DataFrame()
    df["missing_values, %"]=netflix_titles.isnull().sum()*100/len(netflix_titles.isnull())
    df["missing_values, sum"]=netflix_titles.isnull().sum()
    return df.sort_values(by="missing_values, %", ascending=False)
missing_values(netflix_titles)

,"missing_values, %","missing_values, sum"
date_added,0.128419,10
rating,0.089893,7
show_id,0.000000,0
type,0.000000,0
title,0.000000,0
director,0.000000,0
cast,0.000000,0
country,0.000000,0
release_year,0.000000,0
duration_min,0.000000,0


In [45]:
#Print rows which have rating as NaN
netflix_titles[netflix_titles['rating'].isna()].head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration_min,duration_season,listed_in,description
67,s68,Movie,13TH: A Conversation with Oprah Winfrey & Ava ...,Unknown,"Oprah Winfrey, Ava DuVernay",Unknown,2017-01-26,2017,NaN,37,0,Movies,Oprah Winfrey sits down with director Ava DuVe...
2359,s2360,TV Show,Gargantia on the Verdurous Planet,Unknown,"Kaito Ishikawa, Hisako Kanemoto, Ai Kayano, Ka...",Japan,2016-12-01,2013,NaN,0,1,"Anime Series, International TV Shows","After falling through a wormhole, a space-dwel..."
3660,s3661,TV Show,Little Lunch,Unknown,"Flynn Curry, Olivia Deeble, Madison Lu, Oisín ...",Australia,2018-02-01,2015,NaN,0,1,"Kids' TV, TV Comedies","Adopting a child's perspective, this show take..."
3736,s3737,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,2017-04-04,2017,NaN,74,0,Movies,"Louis C.K. muses on religion, eternal love, gi..."
3737,s3738,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,2016-09-16,2010,NaN,84,0,Movies,Emmy-winning comedy writer Louis C.K. brings h...


In [46]:
#Print rows which have date_added as NaN
netflix_titles[netflix_titles['date_added'].isna()].head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration_min,duration_season,listed_in,description
258,s259,TV Show,A Young Doctor's Notebook and Other Stories,Unknown,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaT,2013,TV-MA,0,2,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."
549,s550,TV Show,Anthony Bourdain: Parts Unknown,Unknown,Anthony Bourdain,United States,NaT,2018,TV-PG,0,5,Docuseries,This CNN original series has chef Anthony Bour...
2263,s2264,TV Show,Frasier,Unknown,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",United States,NaT,2003,TV-PG,0,11,"Classic & Cult TV, TV Comedies",Frasier Crane is a snooty but lovable Seattle ...
2288,s2289,TV Show,Friends,Unknown,"Jennifer Aniston, Courteney Cox, Lisa Kudrow, ...",United States,NaT,2003,TV-14,0,10,"Classic & Cult TV, TV Comedies",This hit sitcom follows the merry misadventure...
2555,s2556,TV Show,Gunslinger Girl,Unknown,"Yuuka Nanri, Kanako Mitsuhashi, Eri Sendai, Am...",Japan,NaT,2008,TV-14,0,2,"Anime Series, Crime TV Shows","On the surface, the Social Welfare Agency appe..."


In [47]:
#On further discussion of the use cases, the column show_id is found to be not useful. So drop this column
netflix_titles = netflix_titles.drop(["show_id"], axis = 1)

In [48]:
netflix_titles = netflix_titles.join(netflix_titles['country'].str.split(',', expand=True).add_prefix('country'))
netflix_titles = netflix_titles.join(netflix_titles['listed_in'].str.split(',', expand=True).add_prefix('genre'))
netflix_titles = netflix_titles.applymap(lambda x: x.strip() if isinstance(x, str) else x)
netflix_titles.head()

,type,title,director,cast,country,date_added,release_year,rating,duration_min,duration_season,listed_in,description,country0,country1,country2,country3,country4,country5,country6,country7,country8,country9,country10,country11,genre0,genre1,genre2
0,TV Show,3%,Unknown,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,2020-08-14,2020,TV-MA,0,4,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,Brazil,None,None,None,None,None,None,None,None,None,None,None,International TV Shows,TV Dramas,TV Sci-Fi & Fantasy
1,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,2016-12-23,2016,TV-MA,93,0,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Mexico,None,None,None,None,None,None,None,None,None,None,None,Dramas,International Movies,None
2,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,2018-12-20,2011,R,78,0,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",Singapore,None,None,None,None,None,None,None,None,None,None,None,Horror Movies,International Movies,None
3,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,2017-11-16,2009,PG-13,80,0,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",United States,None,None,None,None,None,None,None,None,None,None,None,Action & Adventure,Independent Movies,Sci-Fi & Fantasy
4,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,2020-01-01,2008,PG-13,123,0,Dramas,A brilliant group of students become card-coun...,United States,None,None,None,None,None,None,None,None,None,None,None,Dramas,None,None


## 2.2 Data cleaning and preparation for Amazon dataset

### 2.2.1 cleaning and preparation of Prime TV Shows Data set.csv

In [49]:
#Reset the display to its default value
pd.reset_option("^display")

In [50]:
#Show the first 5 rows
prime_shows.head()

,S.no.,Name of the show,Year of release,No of seasons available,Language,Genre,IMDb rating,Age of viewers
0,1,Pataal Lok,2020.0,1.0,Hindi,Drama,7.5,18+
1,2,Upload,2020.0,1.0,English,Sci-fi comedy,8.1,16+
2,3,The Marvelous Mrs. Maisel,2017.0,3.0,English,"Drama, Comedy",8.7,16+
3,4,Four More Shots Please,2019.0,2.0,Hindi,"Drama, Comedy",5.3,18+
4,5,Fleabag,2016.0,2.0,English,Comedy,8.7,18+


In [51]:
#View name of the columns
prime_shows.columns

Index(['S.no.', 'Name of the show', 'Year of release',
       'No of seasons available', 'Language', 'Genre', 'IMDb rating',
       'Age of viewers'],
      dtype='object')

In [52]:
#Change the name of the columns
prime_shows.rename(columns={'Name of the show':'Name_of_the_show',
                          'Year of release':'Year_of_release',
                          'No of seasons available':'No_of_seasons',
                           'IMDb rating':'IMDb_rating',
                           'Age of viewers':'Age_of_viewers'}, 
                 inplace=True)

In [53]:
#Show the first 5 rows
prime_shows.head(5)

,S.no.,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,IMDb_rating,Age_of_viewers
0,1,Pataal Lok,2020.0,1.0,Hindi,Drama,7.5,18+
1,2,Upload,2020.0,1.0,English,Sci-fi comedy,8.1,16+
2,3,The Marvelous Mrs. Maisel,2017.0,3.0,English,"Drama, Comedy",8.7,16+
3,4,Four More Shots Please,2019.0,2.0,Hindi,"Drama, Comedy",5.3,18+
4,5,Fleabag,2016.0,2.0,English,Comedy,8.7,18+


In [54]:
# Get more info on the dataset like datatypes, number of records, etc.
prime_shows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 8 columns):
S.no.               404 non-null int64
Name_of_the_show    393 non-null object
Year_of_release     393 non-null float64
No_of_seasons       393 non-null float64
Language            393 non-null object
Genre               393 non-null object
IMDb_rating         182 non-null float64
Age_of_viewers      393 non-null object
dtypes: float64(3), int64(1), object(4)
memory usage: 25.4+ KB


In [55]:
# Get number of null values for each column
prime_shows.isnull().sum()

S.no.                 0
Name_of_the_show     11
Year_of_release      11
No_of_seasons        11
Language             11
Genre                11
IMDb_rating         222
Age_of_viewers       11
dtype: int64

In [56]:
# Get statistical information on the release_year (the only integer column) like percentile, min, max, etc.
prime_shows.describe()

,S.no.,Year_of_release,No_of_seasons,IMDb_rating
count,404.000000,393.000000,393.000000,182.000000
mean,202.500000,2011.279898,2.608142,7.354396
std,116.769003,12.944861,2.592008,0.959372
min,1.000000,1926.000000,1.000000,3.700000
25%,101.750000,2011.000000,1.000000,6.900000
50%,202.500000,2016.000000,2.000000,7.500000
75%,303.250000,2018.000000,3.000000,8.100000
max,404.000000,2020.000000,20.000000,9.000000


In [57]:
#Delete the S.no column, as it is not relevant for our purpose
prime_shows = prime_shows.drop('S.no.', 1)

In [58]:
#Delete rows with all columns as NaN values
prime_shows.dropna(how = "all", inplace=True)
#Show the first and last 5 rows
prime_shows.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,IMDb_rating,Age_of_viewers
0,Pataal Lok,2020.0,1.0,Hindi,Drama,7.5,18+
1,Upload,2020.0,1.0,English,Sci-fi comedy,8.1,16+
2,The Marvelous Mrs. Maisel,2017.0,3.0,English,"Drama, Comedy",8.7,16+
3,Four More Shots Please,2019.0,2.0,Hindi,"Drama, Comedy",5.3,18+
4,Fleabag,2016.0,2.0,English,Comedy,8.7,18+


In [59]:
#Change the rest of NaN values to 0
prime_shows= prime_shows.fillna(0).head(393)

In [60]:
#Converting 'Year of release' and 'No of seasons available' from float to int
prime_shows.No_of_seasons=prime_shows.No_of_seasons.astype(int)
prime_shows.Year_of_release=prime_shows.Year_of_release.astype(int)
prime_shows.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,IMDb_rating,Age_of_viewers
0,Pataal Lok,2020,1,Hindi,Drama,7.5,18+
1,Upload,2020,1,English,Sci-fi comedy,8.1,16+
2,The Marvelous Mrs. Maisel,2017,3,English,"Drama, Comedy",8.7,16+
3,Four More Shots Please,2019,2,Hindi,"Drama, Comedy",5.3,18+
4,Fleabag,2016,2,English,Comedy,8.7,18+


In [61]:
#Count the number of duplicated values in 'Name_of_the_show' column
prime_shows.Name_of_the_show.duplicated().sum()

2

In [62]:
#Show the duplicated records
prime_shows.loc[prime_shows.Name_of_the_show.duplicated(keep=False),:]

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,IMDb_rating,Age_of_viewers
62,The Last Ship,2014,5,English,"Drama, Suspense, Action",7.5,16+
135,The Missing,2014,2,English,Drama,0.0,16+
156,The Missing,2014,2,English,Drama,0.0,16+
222,The Last Ship,2014,5,English,"Drama, Suspense, Action",7.5,16+


In [63]:
#Delete duplicated records
prime_shows=prime_shows.drop_duplicates(subset=['Name_of_the_show',])

In [64]:
# Get more info on the dataset like datatypes, number of records, etc.
prime_shows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 0 to 392
Data columns (total 7 columns):
Name_of_the_show    391 non-null object
Year_of_release     391 non-null int32
No_of_seasons       391 non-null int32
Language            391 non-null object
Genre               391 non-null object
IMDb_rating         391 non-null float64
Age_of_viewers      391 non-null object
dtypes: float64(1), int32(2), object(4)
memory usage: 21.4+ KB


In [65]:
#Sort by highest IMDb rating and show the first 5 records
prime_shows.sort_values(by ='IMDb_rating',ascending = False).head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,IMDb_rating,Age_of_viewers
20,The Test: A New Era for Australia's Team,2020,1,English,Documentary,9.0,16+
281,Seinfeld,1990,9,English,Comedy,8.8,13+
49,Downtown Abbey,2010,6,English,Drama,8.7,7+
37,Hostel Daze,2019,1,Hindi,"Drama, Comedy",8.7,16+
340,The Grand Tour,2017,4,English,Sports,8.7,16+


In [66]:
#Count records = 0 in 'IMDb_rating' column 
(prime_shows['IMDb_rating'] == 0).sum()

210

### 2.2.2 Cleaning and preparation of series_data.csv from iMDB TV Shows

In [67]:
IMDb_shows = pd.read_csv('..\\Datasets\\iMDB_TV_Shows\\series_data.csv')

In [68]:
#Show the first 5 rows
IMDb_shows.head()

,Poster_Link,Series_Title,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre,IMDB_Rating,Overview,Star1,Star2,Star3,Star4,No_of_Votes
0,https://m.media-amazon.com/images/M/MV5BYTRiND...,Game of Thrones,(2011–2019),A,57 min,"Action, Adventure, Drama",9.3,Nine noble families fight for control over the...,Emilia Clarke,Peter Dinklage,Kit Harington,Lena Headey,1773458
1,https://m.media-amazon.com/images/M/MV5BMjhiMz...,Breaking Bad,(2008–2013),18,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,Bryan Cranston,Aaron Paul,Anna Gunn,Betsy Brandt,1468887
2,https://m.media-amazon.com/images/M/MV5BMTc5Zm...,The Walking Dead,(2010– ),18+,44 min,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,Andrew Lincoln,Norman Reedus,Melissa McBride,Danai Gurira,854698
3,https://m.media-amazon.com/images/M/MV5BNDVkYj...,Friends,(1994–2004),13+,22 min,"Comedy, Romance",8.9,Follows the personal and professional lives of...,Jennifer Aniston,Courteney Cox,Lisa Kudrow,Matt LeBlanc,829816
4,https://m.media-amazon.com/images/M/MV5BMjEzMD...,Stranger Things,(2016– ),15,51 min,"Drama, Fantasy, Horror",8.7,"When a young boy disappears, his mother, a pol...",Millie Bobby Brown,Finn Wolfhard,Winona Ryder,David Harbour,824966


In [69]:
# Get more info on the dataset like datatypes, number of records, etc.
IMDb_shows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
Poster_Link            2000 non-null object
Series_Title           2000 non-null object
Runtime_of_Series      2000 non-null object
Certificate            703 non-null object
Runtime_of_Episodes    1957 non-null object
Genre                  2000 non-null object
IMDB_Rating            2000 non-null float64
Overview               2000 non-null object
Star1                  2000 non-null object
Star2                  2000 non-null object
Star3                  1999 non-null object
Star4                  1995 non-null object
No_of_Votes            2000 non-null int64
dtypes: float64(1), int64(1), object(11)
memory usage: 203.2+ KB


In [70]:
#Change the name of the column "Series_Title" to "Name_of_the_show" to be the same as Prime shows dataset
IMDb_shows.rename(columns={'Series_Title':'Name_of_the_show', 'IMDB_Rating':'IMDb_rating'}, 
                 inplace=True)
#Show first 5 rows
IMDb_shows.head()

,Poster_Link,Name_of_the_show,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre,IMDb_rating,Overview,Star1,Star2,Star3,Star4,No_of_Votes
0,https://m.media-amazon.com/images/M/MV5BYTRiND...,Game of Thrones,(2011–2019),A,57 min,"Action, Adventure, Drama",9.3,Nine noble families fight for control over the...,Emilia Clarke,Peter Dinklage,Kit Harington,Lena Headey,1773458
1,https://m.media-amazon.com/images/M/MV5BMjhiMz...,Breaking Bad,(2008–2013),18,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,Bryan Cranston,Aaron Paul,Anna Gunn,Betsy Brandt,1468887
2,https://m.media-amazon.com/images/M/MV5BMTc5Zm...,The Walking Dead,(2010– ),18+,44 min,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,Andrew Lincoln,Norman Reedus,Melissa McBride,Danai Gurira,854698
3,https://m.media-amazon.com/images/M/MV5BNDVkYj...,Friends,(1994–2004),13+,22 min,"Comedy, Romance",8.9,Follows the personal and professional lives of...,Jennifer Aniston,Courteney Cox,Lisa Kudrow,Matt LeBlanc,829816
4,https://m.media-amazon.com/images/M/MV5BMjEzMD...,Stranger Things,(2016– ),15,51 min,"Drama, Fantasy, Horror",8.7,"When a young boy disappears, his mother, a pol...",Millie Bobby Brown,Finn Wolfhard,Winona Ryder,David Harbour,824966


In [71]:
#Sort by IMDb rating from highest to lowest
IMDb_shows=IMDb_shows.sort_values('IMDb_rating', ascending=False)
#Show first 5 rows
IMDb_shows.head()

,Poster_Link,Name_of_the_show,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre,IMDb_rating,Overview,Star1,Star2,Star3,Star4,No_of_Votes
1003,https://m.media-amazon.com/images/M/MV5BMmQ0Yj...,The Chosen,(2017– ),NaN,54 min,"Drama, History",9.7,A charismatic fisherman drowning in debt. A tr...,Shahar Isaac,Jonathan Roumie,Paras Patel,Noah James,13085
483,https://m.media-amazon.com/images/M/MV5BODcwZT...,The Filthy Frank Show,(2011–2017),NaN,12 min,"Comedy, Fantasy, Music",9.5,Filthy Frank and his disease ridden friends ta...,Joji,Artemis Holdenberry,Zeeq,Bloody Hairy,32155
1,https://m.media-amazon.com/images/M/MV5BMjhiMz...,Breaking Bad,(2008–2013),18,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,Bryan Cranston,Aaron Paul,Anna Gunn,Betsy Brandt,1468887
844,https://m.media-amazon.com/images/M/MV5BZTg4NT...,Koombiyo,(2017–2018),NaN,20 min,"Crime, Drama, Thriller",9.5,35 year old bachelor Jehan and his partner in ...,Thumindu Dodantenne,Kalana Gunasekara,Andrew Pulle,Yureni Noshika,16391
182,https://m.media-amazon.com/images/M/MV5BNjgxZT...,Scam 1992: The Harshad Mehta Story,(2020),18,54 min,"Biography, Crime, Drama",9.4,"Set in 1980's and 90's Bombay, it follows the ...",Pratik Gandhi,Shreya Dhanwanthary,Hemant Kher,Anjali Barot,86565


In [72]:
#Total duplicated values in 'Name_of_the_show' column
IMDb_shows.Name_of_the_show.duplicated().sum()

44

In [73]:
#Show the duplicated records
IMDb_shows.loc[IMDb_shows.Name_of_the_show.duplicated(keep=False),:].head()

,Poster_Link,Name_of_the_show,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre,IMDb_rating,Overview,Star1,Star2,Star3,Star4,No_of_Votes
115,https://m.media-amazon.com/images/M/MV5BZmEzN2...,Hagane no renkinjutsushi,(2009–2012),18+,24 min,"Animation, Action, Adventure",9.1,Two brothers search for a Philosopher's Stone ...,Kent Williams,Iemasa Kayumi,Vic Mignogna,Muriel Hofmann,121914
226,https://m.media-amazon.com/images/M/MV5BNTAzMD...,The Twilight Zone,(1959–1964),NaN,51 min,"Drama, Fantasy, Horror",9.0,Ordinary people find themselves in extraordina...,Rod Serling,Robert McCord,Jay Overholts,Vaughn Taylor,71572
1007,https://m.media-amazon.com/images/M/MV5BZDZkZG...,Mahabharat,(1988–1990),12+,45 min,"Adventure, Drama, History",8.9,A dynastic struggle occurs for the throne of H...,Harish Bhimani,Nitish Bharadwaj,Mukesh Khanna,Gajendra Chauhan,12980
270,https://m.media-amazon.com/images/M/MV5BZjNmZD...,Hunter x Hunter,(2011–2014),16,24 min,"Animation, Action, Adventure",8.9,"Gon Freecss aspires to become a Hunter, an exc...",Issei Futamata,Megumi Han,Cristina Valenzuela,Mariya Ise,57950
16,https://m.media-amazon.com/images/M/MV5BMDNkOT...,The Office,(2005–2013),U,22 min,Comedy,8.9,A mockumentary on a group of typical office wo...,Steve Carell,Jenna Fischer,John Krasinski,Rainn Wilson,415367


In [74]:
#Delete duplicates, keep first occurrences which are the highest values
IMDb_shows=IMDb_shows.drop_duplicates(subset=['Name_of_the_show'],keep='first')

In [75]:
#Show the first 5 rows
IMDb_shows.head()

,Poster_Link,Name_of_the_show,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre,IMDb_rating,Overview,Star1,Star2,Star3,Star4,No_of_Votes
1003,https://m.media-amazon.com/images/M/MV5BMmQ0Yj...,The Chosen,(2017– ),NaN,54 min,"Drama, History",9.7,A charismatic fisherman drowning in debt. A tr...,Shahar Isaac,Jonathan Roumie,Paras Patel,Noah James,13085
483,https://m.media-amazon.com/images/M/MV5BODcwZT...,The Filthy Frank Show,(2011–2017),NaN,12 min,"Comedy, Fantasy, Music",9.5,Filthy Frank and his disease ridden friends ta...,Joji,Artemis Holdenberry,Zeeq,Bloody Hairy,32155
1,https://m.media-amazon.com/images/M/MV5BMjhiMz...,Breaking Bad,(2008–2013),18,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,Bryan Cranston,Aaron Paul,Anna Gunn,Betsy Brandt,1468887
844,https://m.media-amazon.com/images/M/MV5BZTg4NT...,Koombiyo,(2017–2018),NaN,20 min,"Crime, Drama, Thriller",9.5,35 year old bachelor Jehan and his partner in ...,Thumindu Dodantenne,Kalana Gunasekara,Andrew Pulle,Yureni Noshika,16391
182,https://m.media-amazon.com/images/M/MV5BNjgxZT...,Scam 1992: The Harshad Mehta Story,(2020),18,54 min,"Biography, Crime, Drama",9.4,"Set in 1980's and 90's Bombay, it follows the ...",Pratik Gandhi,Shreya Dhanwanthary,Hemant Kher,Anjali Barot,86565


In [76]:
# Get more info on the dataset like datatypes, number of records, etc.
IMDb_shows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956 entries, 1003 to 1523
Data columns (total 13 columns):
Poster_Link            1956 non-null object
Name_of_the_show       1956 non-null object
Runtime_of_Series      1956 non-null object
Certificate            692 non-null object
Runtime_of_Episodes    1915 non-null object
Genre                  1956 non-null object
IMDb_rating            1956 non-null float64
Overview               1956 non-null object
Star1                  1956 non-null object
Star2                  1956 non-null object
Star3                  1955 non-null object
Star4                  1951 non-null object
No_of_Votes            1956 non-null int64
dtypes: float64(1), int64(1), object(11)
memory usage: 213.9+ KB


In [77]:
#Merge prime_shows with the additional IMDb_shows dataset
prime_IMDb = pd.merge(prime_shows,IMDb_shows,how="left", on=["Name_of_the_show"])

In [78]:
#Show the first 5 rows
prime_IMDb.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre_x,IMDb_rating_x,Age_of_viewers,Poster_Link,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre_y,IMDb_rating_y,Overview,Star1,Star2,Star3,Star4,No_of_Votes
0,Pataal Lok,2020,1,Hindi,Drama,7.5,18+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Upload,2020,1,English,Sci-fi comedy,8.1,16+,https://m.media-amazon.com/images/M/MV5BZWEyM2...,(2020– ),16,296 min,"Comedy, Mystery, Sci-Fi",8.0,A man is able to choose his own afterlife afte...,Robbie Amell,Andy Allo,Zainab Johnson,Kevin Bigley,40298.0
2,The Marvelous Mrs. Maisel,2017,3,English,"Drama, Comedy",8.7,16+,https://m.media-amazon.com/images/M/MV5BZTFhMD...,(2017– ),16+,57 min,"Comedy, Drama",8.7,A housewife in 1958 decides to become a stand-...,Rachel Brosnahan,Alex Borstein,Michael Zegen,Marin Hinkle,80775.0
3,Four More Shots Please,2019,2,Hindi,"Drama, Comedy",5.3,18+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fleabag,2016,2,English,Comedy,8.7,18+,https://m.media-amazon.com/images/M/MV5BMjA4Mz...,(2016–2019),18+,27 min,"Comedy, Drama",8.7,A comedy series adapted from the award-winning...,Phoebe Waller-Bridge,Sian Clifford,Olivia Colman,Jenny Rainsford,101600.0


In [79]:
#Show the number of null values
prime_IMDb.isnull().sum()

Name_of_the_show         0
Year_of_release          0
No_of_seasons            0
Language                 0
Genre_x                  0
IMDb_rating_x            0
Age_of_viewers           0
Poster_Link            247
Runtime_of_Series      247
Certificate            290
Runtime_of_Episodes    247
Genre_y                247
IMDb_rating_y          247
Overview               247
Star1                  247
Star2                  247
Star3                  247
Star4                  247
No_of_Votes            247
dtype: int64

In [80]:
#Get the percentage of none values 
prime_IMDb.isna().mean().round(4)*100

Name_of_the_show        0.00
Year_of_release         0.00
No_of_seasons           0.00
Language                0.00
Genre_x                 0.00
IMDb_rating_x           0.00
Age_of_viewers          0.00
Poster_Link            63.17
Runtime_of_Series      63.17
Certificate            74.17
Runtime_of_Episodes    63.17
Genre_y                63.17
IMDb_rating_y          63.17
Overview               63.17
Star1                  63.17
Star2                  63.17
Star3                  63.17
Star4                  63.17
No_of_Votes            63.17
dtype: float64

In [81]:
#Delete the columns that are not relevant for our purpose
prime_IMDb= prime_IMDb.drop(['Poster_Link', 'Runtime_of_Series', 'Certificate','Runtime_of_Episodes','Genre_y', 'Overview','Star1','Star2', 'Star3', 'Star4', 'No_of_Votes'], 1)
#Show first 5 rows
prime_IMDb.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre_x,IMDb_rating_x,Age_of_viewers,IMDb_rating_y
0,Pataal Lok,2020,1,Hindi,Drama,7.5,18+,NaN
1,Upload,2020,1,English,Sci-fi comedy,8.1,16+,8.0
2,The Marvelous Mrs. Maisel,2017,3,English,"Drama, Comedy",8.7,16+,8.7
3,Four More Shots Please,2019,2,Hindi,"Drama, Comedy",5.3,18+,NaN
4,Fleabag,2016,2,English,Comedy,8.7,18+,8.7


In [82]:
#Fill NA records from IMBb_rating_y with IMBb_rating_x records
prime_IMDb['IMDb_rating_y']=prime_IMDb['IMDb_rating_y'].fillna(prime_IMDb['IMDb_rating_x'])

In [83]:
#Show first 5 rows
prime_IMDb.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre_x,IMDb_rating_x,Age_of_viewers,IMDb_rating_y
0,Pataal Lok,2020,1,Hindi,Drama,7.5,18+,7.5
1,Upload,2020,1,English,Sci-fi comedy,8.1,16+,8.0
2,The Marvelous Mrs. Maisel,2017,3,English,"Drama, Comedy",8.7,16+,8.7
3,Four More Shots Please,2019,2,Hindi,"Drama, Comedy",5.3,18+,5.3
4,Fleabag,2016,2,English,Comedy,8.7,18+,8.7


In [84]:
#Sort by IMDb_rating_y from highest to lowest
prime_IMDb=prime_IMDb.sort_values('IMDb_rating_y', ascending=False)
#Show first 5 records
prime_IMDb.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre_x,IMDb_rating_x,Age_of_viewers,IMDb_rating_y
20,The Test: A New Era for Australia's Team,2020,1,English,Documentary,9.0,16+,9.0
258,The Office,2005,9,English,Comedy,0.0,13+,8.9
279,Seinfeld,1990,9,English,Comedy,8.8,13+,8.8
338,The Grand Tour,2017,4,English,Sports,8.7,16+,8.7
4,Fleabag,2016,2,English,Comedy,8.7,18+,8.7


In [85]:
#Delete the 'IMDb_rating_x' column, as we do not need it anymore
prime_IMDb = prime_IMDb.drop('IMDb_rating_x', 1)
#Show the first 5 rows
prime_IMDb.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre_x,Age_of_viewers,IMDb_rating_y
20,The Test: A New Era for Australia's Team,2020,1,English,Documentary,16+,9.0
258,The Office,2005,9,English,Comedy,13+,8.9
279,Seinfeld,1990,9,English,Comedy,13+,8.8
338,The Grand Tour,2017,4,English,Sports,16+,8.7
4,Fleabag,2016,2,English,Comedy,18+,8.7


In [86]:
#Change the name of the column from 'IMDb_rating_y' to 'IMDb_rating' and 'Genre_x' to 'Genre.
prime_IMDb.rename(columns={'IMDb_rating_y':'IMDb_rating', 'Genre_x':'Genre'}, 
                 inplace=True)
#Show the first 5 rows
prime_IMDb.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,Age_of_viewers,IMDb_rating
20,The Test: A New Era for Australia's Team,2020,1,English,Documentary,16+,9.0
258,The Office,2005,9,English,Comedy,13+,8.9
279,Seinfeld,1990,9,English,Comedy,13+,8.8
338,The Grand Tour,2017,4,English,Sports,16+,8.7
4,Fleabag,2016,2,English,Comedy,18+,8.7


In [87]:
#Count records = 0 in IMDb_rating column 
(prime_IMDb['IMDb_rating'] == 0).sum()

153

### 2.2.3 Cleaning and preparation of series_manual.csv of IMDb TV Shows_Manual

In [88]:
IMDb_manual = pd.read_csv('..\\Datasets\\iMDB_TV_Shows\\series_manual.csv')

In [89]:
#Show the first 5 rows
IMDb_manual.head()

,Name_of_the_show,IMDb_rating
0,Atchoo,NaN
1,WassUp With You,NaN
2,Loudermilk,7.7
3,The 2018 Rose Parade Hosted by Cord & Tish,7.6
4,The Best of Groucho,NaN


In [90]:
#Merge prime_shows with the additional IMDb_shows dataset
prime_IMDb_final = pd.merge(prime_IMDb,IMDb_manual,how="left", on=["Name_of_the_show"])

In [91]:
#Show the first 5 rows
prime_IMDb_final.head(5)

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,Age_of_viewers,IMDb_rating_x,IMDb_rating_y
0,The Test: A New Era for Australia's Team,2020,1,English,Documentary,16+,9.0,NaN
1,The Office,2005,9,English,Comedy,13+,8.9,NaN
2,Seinfeld,1990,9,English,Comedy,13+,8.8,NaN
3,The Grand Tour,2017,4,English,Sports,16+,8.7,NaN
4,Fleabag,2016,2,English,Comedy,18+,8.7,NaN


In [92]:
#Show the last 5 rows
prime_IMDb_final.tail(5)

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,Age_of_viewers,IMDb_rating_x,IMDb_rating_y
386,Journey with Dylan Dreyer,2017,1,English,Action,13+,0.0,7.9
387,Vir the Robot Boy,2013,2,Hindi,Kids,7+,0.0,2.8
388,Forensic Files,1998,20,English,Documentary,13+,0.0,8.8
389,Highway on my Plate,2013,1,English,Adventure,All,0.0,NaN
390,Gina Brillon: The Floor is Lava,2020,1,English,Comedy,16+,0.0,6.1


In [93]:
#Fill NA records from IMBb_rating_y with IMBb_rating_x records
prime_IMDb_final['IMDb_rating_y']=prime_IMDb_final['IMDb_rating_y'].fillna(prime_IMDb_final['IMDb_rating_x'])

In [94]:
#Delete the 'IMDb_rating_x' column, as we do not need it anymore
prime_IMDb_final = prime_IMDb_final.drop('IMDb_rating_x', 1)

In [95]:
#Change the name of the column from IMDb_rating_y to IMDb_rating
prime_IMDb_final.rename(columns={'IMDb_rating_y':'IMDb_rating'}, 
                 inplace=True)
#Show the first 5 rows
prime_IMDb_final.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,Age_of_viewers,IMDb_rating
0,The Test: A New Era for Australia's Team,2020,1,English,Documentary,16+,9.0
1,The Office,2005,9,English,Comedy,13+,8.9
2,Seinfeld,1990,9,English,Comedy,13+,8.8
3,The Grand Tour,2017,4,English,Sports,16+,8.7
4,Fleabag,2016,2,English,Comedy,18+,8.7


In [96]:
#Count records = 0 in IMDb_rating column 
(prime_IMDb_final['IMDb_rating'] == 0).sum()

38

In [97]:
#Count the number of series by Genre
prime_IMDb_final['Genre'].value_counts()

Drama                                         124
Comedy                                         97
Kids                                           29
Action                                         26
Drama, Comedy                                  15
Drama, Suspense                                12
Arts, Entertainment, Culture                   12
Drama, Action                                  11
Drama, Sci-fi                                   7
Sci-fi                                          4
Drama, Romance                                  4
Drama, Action, Suspense                         3
Documentary                                     3
Drama, Documentary                              2
Drama, Horror                                   2
Drama, Fantasy                                  2
Drama, Horror, Suspense                         2
Animation                                       2
Adventure                                       2
Kids, Animation                                 2


In [98]:
#Replacing Sci-fi comedy for Sci-fi, Comedy
prime_IMDb_final['Genre'] = np.where(prime_IMDb_final['Genre'] == 'Sci-fi comedy' , 'Sci-fi, Comedy', prime_IMDb_final['Genre'])

In [99]:
#Split the column 'Genre' into multiple columns
prime_IMDb_final = prime_IMDb_final.join(prime_IMDb_final['Genre'].str.split(',', expand=True).add_prefix('Genre'))

# Naturally, due to the column split, there is a possibility of padded spaces in the values.
# Stripping the extra spaces throuout the dataset
prime_IMDb_final = prime_IMDb_final.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [100]:
#Show the first 5 rows
prime_IMDb_final.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Genre,Age_of_viewers,IMDb_rating,Genre0,Genre1,Genre2,Genre3,Genre4
0,The Test: A New Era for Australia's Team,2020,1,English,Documentary,16+,9.0,Documentary,None,None,None,None
1,The Office,2005,9,English,Comedy,13+,8.9,Comedy,None,None,None,None
2,Seinfeld,1990,9,English,Comedy,13+,8.8,Comedy,None,None,None,None
3,The Grand Tour,2017,4,English,Sports,16+,8.7,Sports,None,None,None,None
4,Fleabag,2016,2,English,Comedy,18+,8.7,Comedy,None,None,None,None


In [101]:
#Get the percentage of none values 
prime_IMDb_final.isna().mean().round(4)*100

Name_of_the_show     0.00
Year_of_release      0.00
No_of_seasons        0.00
Language             0.00
Genre                0.00
Age_of_viewers       0.00
IMDb_rating          0.00
Genre0               0.00
Genre1              74.42
Genre2              91.56
Genre3              98.98
Genre4              99.74
dtype: float64

In [102]:
#Delete the "Genre" columns which percentaje of none values is too high.
#We will keep just the first 3 genres
prime_IMDb_final= prime_IMDb_final.drop(['Genre', 'Genre3', 'Genre4'], 1)
prime_IMDb_final.head()

,Name_of_the_show,Year_of_release,No_of_seasons,Language,Age_of_viewers,IMDb_rating,Genre0,Genre1,Genre2
0,The Test: A New Era for Australia's Team,2020,1,English,16+,9.0,Documentary,None,None
1,The Office,2005,9,English,13+,8.9,Comedy,None,None
2,Seinfeld,1990,9,English,13+,8.8,Comedy,None,None
3,The Grand Tour,2017,4,English,16+,8.7,Sports,None,None
4,Fleabag,2016,2,English,18+,8.7,Comedy,None,None


In [103]:
#Change "Comeddy" and "comedy" to Comedy 
prime_IMDb_final.loc[prime_IMDb_final['Genre0'] == 'Comeddy', 'Genre0'] = 'Comedy' 
prime_IMDb_final.loc[prime_IMDb_final['Genre0'] == 'comedy', 'Genre0'] = 'Comedy'
prime_IMDb_final.loc[prime_IMDb_final['Genre1'] == 'Comeddy', 'Genre1'] = 'Comedy' 
prime_IMDb_final.loc[prime_IMDb_final['Genre1'] == 'comedy', 'Genre1'] = 'Comedy'
prime_IMDb_final.loc[prime_IMDb_final['Genre2'] == 'Comeddy', 'Genre2'] = 'Comedy' 
prime_IMDb_final.loc[prime_IMDb_final['Genre2'] == 'comedy', 'Genre2'] = 'Comedy'

In [104]:
#Count the number of series by Genre
prime_IMDb_final['Genre0'].value_counts()

Drama          199
Comedy         100
Kids            31
Action          27
Arts            12
Sci-fi           8
Sports           3
Animation        3
Documentary      3
Adventure        2
Fantasy          2
Horror           1
Name: Genre0, dtype: int64

## 2.3 Data cleaning and preparation for IMDB

In [105]:
# Merge the imdb movies and ratings datasets into one
imdb = pd.merge(imdb_movies, imdb_ratings, on = "imdb_title_id", how = "outer")
imdb.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0


In [106]:
imdb.shape

(85855, 70)

In [107]:
# Set the index of the dataframe as imdb_title_id
imdb.set_index('imdb_title_id', inplace = True)

#### Check duplicates

In [108]:
# Check if there are any duplicate rows
imdb[imdb.duplicated()== True]

,title,original_title,year,date_published,genre,duration,country,language,director,writer,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
imdb_title_id,,,,,,,,,,,,,,,,,,,,,


> There are no duplicates

#### Drop unnecessary columns

> * We don't need the original_title columns of the movies, because we already have title. 
> * Similarly we also will not be using description
> * For our analysis, we will be considering only weighted average vote. i.e. we don't need other avg vote columns
> * There is already a column named votes present representing the total_votes. Therefore dropping redundant column total_votes.
> * We will also not be considering the number of votes for each rating category. Therefore dropping votes_10-votes_1

In [109]:
# Remove the unnecessary
imdb = imdb.drop(['original_title', 'description', 'date_published', 'avg_vote', 'mean_vote', 'median_vote', 'total_votes',
                  'votes_10', 'votes_9', 'votes_8', 'votes_7', 'votes_6', 'votes_5', 'votes_4', 'votes_3', 
                  'votes_2', 'votes_1'], axis = 1)

#### Check if any column has only one value throught

In [110]:
imdb.loc[:,imdb.nunique()==1].columns

Index([], dtype='object')

> None of the columns exhibit this. Hence, retaining all columns.

#### Imputing null values

In [111]:
# Inspect the null value percentage in a column
print(round(100*(imdb.isnull().sum()/len(imdb.index)), 2))

title                         0.00
year                          0.00
genre                         0.00
duration                      0.00
country                       0.07
language                      0.97
director                      0.10
writer                        1.83
production_company            5.19
actors                        0.08
votes                         0.00
budget                       72.38
usa_gross_income             82.15
worlwide_gross_income        63.87
metascore                    84.50
reviews_from_users            8.85
reviews_from_critics         13.74
weighted_average_vote         0.00
allgenders_0age_avg_vote     61.14
allgenders_0age_votes        61.14
allgenders_18age_avg_vote     0.82
allgenders_18age_votes        0.82
allgenders_30age_avg_vote     0.01
allgenders_30age_votes        0.01
allgenders_45age_avg_vote     0.09
allgenders_45age_votes        0.09
males_allages_avg_vote        0.00
males_allages_votes           0.00
males_0age_avg_vote 

> We will first remove the columns that have more than 60% missing values, but before that let's explore if there is any possibility to impute the missing values from total votes

In [112]:
# Check if one of the columns have with null values can be imputed from the other 2 non null values
imdb.loc[imdb['allgenders_0age_votes'].isnull() & imdb['males_0age_votes'].notnull() & imdb['females_0age_votes'].notnull()]

,title,year,genre,duration,country,language,director,writer,production_company,actors,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
imdb_title_id,,,,,,,,,,,,,,,,,,,,,


In [113]:
# Check if all the null values of these 3 columns are for the same record
print(imdb.loc[imdb['allgenders_0age_votes'].isnull() & imdb['males_0age_votes'].isnull() & imdb['females_0age_votes'].isnull()].shape)
print(imdb.loc[imdb['allgenders_0age_votes'].isnull()].shape)
print(imdb.loc[imdb['males_0age_votes'].isnull()].shape)
print(imdb.loc[imdb['females_0age_votes'].isnull()].shape)

(52496, 52)
(52496, 52)
(58444, 52)
(63738, 52)


In [114]:
# Check if one of the columns have with null values can be imputed from the other 2 non null values
# Check if all the null values of these 3 columns are for the same record
print(imdb.loc[imdb['allgenders_0age_avg_vote'].isnull() & imdb['males_0age_avg_vote'].isnull() & imdb['females_0age_avg_vote'].isnull()].shape)
print(imdb.loc[imdb['allgenders_0age_avg_vote'].isnull()].shape)
print(imdb.loc[imdb['males_0age_avg_vote'].isnull()].shape)
print(imdb.loc[imdb['females_0age_avg_vote'].isnull()].shape)

(52496, 52)
(52496, 52)
(58444, 52)
(63738, 52)


> 1. We can conclude that even if we try imputing, it can be done to a very less percentage
> 2. Therefore, we can drop these columns

In [115]:
imdb = imdb.drop(['allgenders_0age_votes', 'allgenders_0age_avg_vote', 'males_0age_votes', 'males_0age_avg_vote', 
                  'females_0age_votes', 'females_0age_avg_vote'], axis = 1)

In [116]:
# Similarly other columns with more than 70% null values can be dropped, 
# because imputing them will modify the true meaning of the data
imdb = imdb.drop(['budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore'], axis = 1)

In [117]:
# Inspect the columns for null value percentage again
print(round(100*(imdb.isnull().sum()/len(imdb.index)), 2))

title                         0.00
year                          0.00
genre                         0.00
duration                      0.00
country                       0.07
language                      0.97
director                      0.10
writer                        1.83
production_company            5.19
actors                        0.08
votes                         0.00
reviews_from_users            8.85
reviews_from_critics         13.74
weighted_average_vote         0.00
allgenders_18age_avg_vote     0.82
allgenders_18age_votes        0.82
allgenders_30age_avg_vote     0.01
allgenders_30age_votes        0.01
allgenders_45age_avg_vote     0.09
allgenders_45age_votes        0.09
males_allages_avg_vote        0.00
males_allages_votes           0.00
males_18age_avg_vote          1.71
males_18age_votes             1.71
males_30age_avg_vote          0.01
males_30age_votes             0.01
males_45age_avg_vote          0.12
males_45age_votes             0.12
females_allages_avg_

> 1. Clearly, the null values for are in the order all_genders_votes < males_avg_votes < females_avg_votes.
> 2. We can fill in the null values if we have values in any of the two columns with the help of the average formula:
                 (males_avg_votes + females_avg_votes)/2 = all_genders_avg_votes

In [118]:
# First try to compute for females_18age_avg_votes, where the average votes for men and all genders are known.
imdb['females_18age_avg_vote'] = np.where(imdb['allgenders_18age_avg_vote'].notnull() & 
         imdb['males_18age_avg_vote'].notnull() & 
         imdb['females_18age_avg_vote'].isnull(), 
         (2*imdb['allgenders_18age_avg_vote'] - imdb['males_18age_avg_vote']), imdb['females_18age_avg_vote'])

In [119]:
# Similarly, compute for males_age18_avg_votes
imdb['males_18age_avg_vote'] = np.where(imdb['allgenders_18age_avg_vote'].notnull() & 
         imdb['males_18age_avg_vote'].isnull() & 
         imdb['females_18age_avg_vote'].notnull(), 
         (2*imdb['allgenders_18age_avg_vote'] - imdb['females_18age_avg_vote']), imdb['males_18age_avg_vote'])

In [120]:
# Similarly, compute for females_30age_avg_votes
imdb['females_30age_avg_vote'] = np.where(imdb['allgenders_30age_avg_vote'].notnull() & 
         imdb['males_30age_avg_vote'].notnull() & 
         imdb['females_30age_avg_vote'].isnull(), 
         (2*imdb['allgenders_30age_avg_vote'] - imdb['males_30age_avg_vote']), imdb['females_30age_avg_vote'])

In [121]:
# Similarly, compute for males_30age_avg_votes
imdb['males_30age_avg_vote'] = np.where(imdb['allgenders_30age_avg_vote'].notnull() & 
         imdb['females_30age_avg_vote'].notnull() & 
         imdb['males_30age_avg_vote'].isnull(), 
         (2*imdb['allgenders_30age_avg_vote'] - imdb['females_30age_avg_vote']), imdb['males_30age_avg_vote'])

In [122]:
# Similarly, compute for females_45age_avg_votes
imdb['females_45age_avg_vote'] = np.where(imdb['allgenders_45age_avg_vote'].notnull() & 
         imdb['males_45age_avg_vote'].notnull() & 
         imdb['females_45age_avg_vote'].isnull(), 
         (2*imdb['allgenders_45age_avg_vote'] - imdb['males_45age_avg_vote']), imdb['females_45age_avg_vote'])

In [123]:
# Similarly, compute for males_45age_avg_votes
imdb['males_45age_avg_vote'] = np.where(imdb['allgenders_45age_avg_vote'].notnull() & 
         imdb['females_45age_avg_vote'].notnull() & 
         imdb['males_45age_avg_vote'].isnull(), 
         (2*imdb['allgenders_45age_avg_vote'] - imdb['females_45age_avg_vote']), imdb['males_45age_avg_vote'])

> * Similarly, we can now calculate the females and males votes from allgenders votes as,
                          allgenders_votes = males_votes + females_votes

In [124]:
# # Compute for females_18age_votes
imdb['females_18age_votes'] = np.where(imdb['allgenders_18age_votes'].notnull() & 
         imdb['males_18age_votes'].notnull() & 
         imdb['females_18age_votes'].isnull(), 
         (imdb['allgenders_18age_votes'] - imdb['males_18age_votes']), imdb['females_18age_votes'])

In [125]:
# # Compute for males_18age_votes
imdb['males_18age_votes'] = np.where(imdb['allgenders_18age_votes'].notnull() & 
         imdb['females_18age_votes'].notnull() & 
         imdb['males_18age_votes'].isnull(), 
         (imdb['allgenders_18age_votes'] - imdb['females_18age_votes']), imdb['males_18age_votes'])

In [126]:
# # Compute for females_30age_votes
imdb['females_30age_votes'] = np.where(imdb['allgenders_30age_votes'].notnull() & 
         imdb['males_30age_votes'].notnull() & 
         imdb['females_30age_votes'].isnull(), 
         (imdb['allgenders_30age_votes'] - imdb['males_30age_votes']), imdb['females_30age_votes'])

In [127]:
# # Compute for females_45age_votes
imdb['females_45age_votes'] = np.where(imdb['allgenders_45age_votes'].notnull() & 
         imdb['males_45age_votes'].notnull() & 
         imdb['females_45age_votes'].isnull(), 
         (imdb['allgenders_45age_votes'] - imdb['males_45age_votes']), imdb['females_45age_votes'])

In [128]:
# # Compute for males_45age_votes
imdb['males_45age_votes'] = np.where(imdb['allgenders_45age_votes'].notnull() & 
         imdb['females_45age_votes'].notnull() & 
         imdb['males_45age_votes'].isnull(), 
         (imdb['allgenders_45age_votes'] - imdb['females_45age_votes']), imdb['males_45age_votes'])

> * Similarly, we can now calculate the reviews from users and critics from votes as,
                        votes = reviews_from_users + reviews_from_critics

In [129]:
# # Compute for reviews_from_critics
imdb['reviews_from_critics'] = np.where(imdb['votes'].notnull() & 
         imdb['reviews_from_users'].notnull() & 
         imdb['reviews_from_critics'].isnull(), 
         (imdb['votes'] - imdb['reviews_from_users']), imdb['reviews_from_critics'])

In [130]:
# # Compute for reviews_from_users
imdb['reviews_from_users'] = np.where(imdb['votes'].notnull() & 
         imdb['reviews_from_critics'].notnull() & 
         imdb['reviews_from_users'].isnull(), 
         (imdb['votes'] - imdb['reviews_from_critics']), imdb['reviews_from_users'])

In [131]:
# Inspect the columns for null value percentage again
print(round(100*(imdb.isnull().sum()/len(imdb.index)), 2))

title                        0.00
year                         0.00
genre                        0.00
duration                     0.00
country                      0.07
language                     0.97
director                     0.10
writer                       1.83
production_company           5.19
actors                       0.08
votes                        0.00
reviews_from_users           4.08
reviews_from_critics         4.08
weighted_average_vote        0.00
allgenders_18age_avg_vote    0.82
allgenders_18age_votes       0.82
allgenders_30age_avg_vote    0.01
allgenders_30age_votes       0.01
allgenders_45age_avg_vote    0.09
allgenders_45age_votes       0.09
males_allages_avg_vote       0.00
males_allages_votes          0.00
males_18age_avg_vote         0.82
males_18age_votes            0.82
males_30age_avg_vote         0.01
males_30age_votes            0.01
males_45age_avg_vote         0.10
males_45age_votes            0.10
females_allages_avg_vote     0.09
females_allage

> Now that we have taken care of most of the null values for numerical columns, we will now replace null values in categorical columns with 'unknown'

In [132]:
imdb.fillna({'country':'Unknown', 'language':'Unknown', 'director':'Unknown', 'writer': 'Unknown', 
             'production_company': 'Unknown', 'actors':'Unknown', }, inplace=True)

> * Let's now ensure that all the columns have valid values.
> * Starting with the column 'year'.

#### Cleaning the column year

In [133]:
# Check the unique values in year and then convert to numeric if everything is okay
imdb.year.unique()

array([1894, 1906, 1911, 1912, 1919, 1913, 1914, 1915, 1916, 1917, 1918,
       1920, 1921, 1924, 1922, 1923, 1925, 1926, 1935, 1927, 1928, 1983,
       1929, 1930, 1932, 1931, 1937, 1938, 1933, 1934, 1936, 1940, 1939,
       1942, 1943, 1941, 1948, 1944, 2001, 1946, 1945, 1947, 1973, 1949,
       1950, 1952, 1951, 1962, 1953, 1954, 1955, 1961, 1956, 1958, 1957,
       1959, 1960, 1963, 1965, 1971, 1964, 1966, 1968, 1967, 1969, 1976,
       1970, 1979, 1972, 1981, 1978, 2000, 1989, 1975, 1974, 1986, 1990,
       2018, 1977, 1982, 1980, 1993, 1984, 1985, 1988, 1987, 2005, 1991,
       2002, 1994, 1992, 1995, 2017, 1997, 1996, 2006, 1999, 1998, 2007,
       2008, 2003, 2004, 2010, 2009, 2011, 2013, 2012, 2016, 2015, 2014,
       2019, 2020, '2012', '2015', '2009', '2013', '2018', '2014', '2017',
       '2011', '2016', '1981', '1975', '2010', '1984', '2007', '2006',
       '2001', '2004', '1979', '2019', '1967', '1978', '2003', '2005',
       '1969', '1990', '1983', '2002', '1996', '2008'

In [134]:
# Clearly, there is a need to replace 'TV Movie 2019' to 2019. Then convert the column to integer
imdb['year'] = np.where(imdb['year'] == 'TV Movie 2019' , '2019', imdb['year'])

# Converting year column to integer
imdb['year'] = pd.to_numeric(imdb['year'])

#### Check the column 'genre'
> * The column genre has multiple genres for the same movie. Therefore, split them into multiple columns in order to get a clear picture while analyzing the data in the future.

In [135]:
imdb = imdb.join(imdb['genre'].str.split(',', expand=True).add_prefix('genre'))

#### Check the column 'duration'

In [136]:
# Check out the unique columns
imdb.duration.unique()

array([ 45,  70,  53, 100,  68,  60,  85, 120,  55, 121,  54,  96,  61,
        90,  50,  88,  72,  78, 148,  52, 124,  59,  63,  84,  65,  81,
       199,  74,  80,  82,  67,  56, 195,  77,  71,  46, 421, 105,  57,
        58,  73,  64,  62, 163, 300, 116,  69, 125,  97, 138, 112,  91,
        87,  48,  83, 136,  75,  94, 117,  93,  76, 418,  86, 223,  99,
       122, 207, 166,  95,  92,  66, 106, 145, 167,  79, 107, 109, 104,
        89, 102, 150, 131,  47, 119,  98, 110, 143, 137, 128, 101, 183,
       133, 135, 111, 141, 115, 153, 129, 123, 108, 155, 151, 170, 140,
       113, 114, 250, 132, 118, 160, 240, 142, 103, 144,  49, 127, 156,
       130, 165, 147, 152, 226, 227, 231, 218, 200, 210, 225, 243,  51,
       281, 208, 306, 257, 245, 255, 328, 168, 269, 303, 290, 212, 224,
       126, 260, 299, 264, 149, 211, 302, 238, 258, 263, 215, 265, 205,
       237, 220, 285, 293, 267, 216, 241,  43, 134, 184, 261, 197, 244,
       146, 177, 196, 154, 189, 219, 201, 180, 159, 192, 139, 17

> * Clearly, this column is clean and we can see that all the values are in minutes.
#### Check the column country

In [137]:
# Check out the unique values
print(imdb.country.unique())

['USA' 'Australia' 'Germany, Denmark' ...
 'Iran, France, Germany, Switzerland, Luxembourg, Lebanon'
 'Spain, Portugal, Argentina' 'India, Netherlands, Singapore']


> * We can see that the movies released in multiple countries listed in the same column, separated by comma.
> * We cannot split like how we did for genre, because a movie could have been released in more than 20 countries.
> * We can handle this by extracting just the first 3 countries from the list and then create a separate column indicating the total number of countires the movie was released in.

In [138]:
# Add a column indicating the total number of countries it was released in.
imdb['total_countries'] = imdb['country'].apply(lambda x: len(x.split(',')))

In [139]:
# Now extract first 3 countries
imdb['country0'] = imdb['country'].str.split(',').str[0]
imdb['country1'] = imdb['country'].str.split(',').str[1]
imdb['country2'] = imdb['country'].str.split(',').str[2]

#### Check the column language

In [140]:
imdb.language.unique()

array(['None', 'Unknown', 'English', ..., 'Persian, Urdu',
       'English, Swiss German, German',
       'English, Polish, Russian, German'], dtype=object)

> * First, we need to replace 'None' with 'Unknown'
> * We can use the same strategy we used for the column country

In [141]:
# Replace None with Unknown
imdb.language = imdb.language.replace('None', 'Unknown')

In [142]:
# Similarly, there are values in language like 'None, English' or 'English, None'.
# Replace None in these scenarios as well with empty string
imdb['language'] = imdb['language'].str.replace('None,', '')
imdb['language'] = imdb['language'].str.replace(', None', '')

> * We can replace language that has a value of 'Unknown' with the official language of the corresponding country. Since a country can have many official languages, we replace only for those countries that have only one official language, and those movies that have been released in only one country
> * We can get the list of official languages by country from wikipedia

In [143]:
# Scrape Wikipedia list of official languages by country
# Extracte the required table
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory').read()
soup = bs.BeautifulSoup(source,'lxml')
table = soup.find('table', attrs={'class':'wikitable sortable'})

#Store the table extracted table in dataframe
country_details = pd.read_html(str(table))[0]
country_details.head()

,Country,Official language,Regional language,Minority language,National language,Widely spoken
0,Abkhazia[a],Abkhaz Russian,NaN,Georgian,Abkhaz,NaN
1,Afghanistan[1],Pashto Dari,Uzbek[b] Turkmen[b] Pashai[b] Nuristani[b] Bal...,NaN,Pashto,NaN
2,Albania[2],Albanian,NaN,Greek Macedonian Aromanian,NaN,Italian
3,Algeria[3],Arabic Tamazight,NaN,NaN,Arabic Tamazight,French
4,Andorra,Catalan[4],NaN,Spanish French Portuguese,NaN,NaN


In [144]:
# Clearly, citation is also extracted. Use regex to remove them
country_details.replace('(\[.*?\]|\(.*?\))', '', regex = True, inplace = True)
# Replace 'United Kingdom and Crown dependencies etc.' with 'UK' ans 'United States' with 'USA'
country_details.replace('United Kingdom and Crown dependencies etc.', 'UK', inplace = True)
country_details.replace('United States', 'USA', inplace = True)

In [145]:
# Merge imdb and country_details to get the official language
imdb = imdb.merge(country_details[['Official language', 'Country']], how = 'left', left_on = 'country0', right_on = 'Country')

In [146]:
# Replace the values 'Únknown' values of language wherever applicable
imdb.language = np.where((imdb.language == 'Unknown') & (imdb.country1.isnull()) & (imdb['Official language'].notnull()) 
         & (~imdb['Official language'].str.contains(r'\s+', na=False)), imdb['Official language'], imdb.language)

In [147]:
# Add a column indicating the total number of languages it was released in.
imdb['total_languages'] = imdb['language'].apply(lambda x: len(x.split(',')))

In [148]:
# Now extract first 3 languages
imdb['language0'] = imdb['language'].str.split(',').str[0]
imdb['language1'] = imdb['language'].str.split(',').str[1]
imdb['language2'] = imdb['language'].str.split(',').str[2]

#### Check the column director

In [149]:
# A movie would have been directed by more than one director.
# Split the column like we did for genre
imdb = imdb.join(imdb['director'].str.split(',', expand=True).add_prefix('director'))
imdb.head()

,title,year,genre,duration,country,language,director,writer,production_company,actors,...,country1,country2,Official language,Country,total_languages,language0,language1,language2,director0,director1
0,Miss Jerry,1894,Romance,45,USA,Unknown,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",...,NaN,NaN,None,USA,1,Unknown,NaN,NaN,Alexander Black,None
1,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia,Unknown,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",...,NaN,NaN,None,Australia,1,Unknown,NaN,NaN,Charles Tait,None
2,Den sorte drøm,1911,Drama,53,"Germany, Denmark",Unknown,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",...,Denmark,NaN,German,Germany,1,Unknown,NaN,NaN,Urban Gad,None
3,Cleopatra,1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",...,NaN,NaN,None,USA,1,English,NaN,NaN,Charles L. Gaskill,None
4,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",...,NaN,NaN,Italian,Italy,1,Italian,NaN,NaN,Francesco Bertolini,Adolfo Padovan


> * Clearly there have been atmost 2 directors directing a movie together.
#### Clean the column writer

In [150]:
# Split the column writer
imdb = imdb.join(imdb['writer'].str.split(',', expand=True).add_prefix('writer'))
imdb.head(2)

,title,year,genre,duration,country,language,director,writer,production_company,actors,...,Official language,Country,total_languages,language0,language1,language2,director0,director1,writer0,writer1
0,Miss Jerry,1894,Romance,45,USA,Unknown,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",...,None,USA,1,Unknown,NaN,NaN,Alexander Black,None,Alexander Black,None
1,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia,Unknown,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",...,None,Australia,1,Unknown,NaN,NaN,Charles Tait,None,Charles Tait,None


> * Similarly, we can see that a movie has been co written by atmost 2 authors

#### Check the column actors
> * A movie is bound to have multiple actors. We can extract the first 3 actors from the list like we did for countries and languages.
> * Sometimes a movie can also star more than 3 famous actors. Hence we can again keep a count of the number of actors by creating a new indicator - total actors

In [151]:
# Add a column indicating the total number of actors.
imdb['total_actors'] = imdb['actors'].apply(lambda x: len(x.split(',')))

In [152]:
# Now extract first 3 actors
imdb['actors0'] = imdb['actors'].str.split(',').str[0]
imdb['actors1'] = imdb['actors'].str.split(',').str[1]
imdb['actors2'] = imdb['actors'].str.split(',').str[2]

#### Final cleanup

In [153]:
# Finally drop all the unnecessary columns from the cleaned dataset
imdb.drop(['Official language', 'Country'], axis=1, inplace=True)

In [154]:
# Naturally, due to the column split, there is a possibility of padded spaces in the values.
# Strip the extra spaces throuout the dataset
imdb = imdb.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [155]:
# Rename votes column with total_votes_imdb to avoid confusions and better readability
imdb.rename(columns={"votes": "total_votes_imdb"}, inplace = True)

In [156]:
# The genre 'music' an 'musical' are similar. Replace 'music' with 'musical' in the dataset
imdb.replace('Music', 'Musical', inplace = True)

In [157]:
imdb.shape

(85855, 61)

> Now that we have clean data, we can merge it with rotten tomatoes dataset to get complete movie information

## 2.4 Data cleaning and preparation for Rotten Tomatoes Dataset

In [158]:
# combine the two datasets movies and critics
rt = rt_movies.merge(rt_critics, on = 'rotten_tomatoes_link')

#### Find and Remove the duplicated values

In [159]:
## find the total number of duplicate values
rt.rotten_tomatoes_link.duplicated().sum()

1112181

In [160]:
#show the duplicated records
rt.loc[rt.rotten_tomatoes_link.duplicated(keep=False),:].head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,43,73,76,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,43,73,76,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,43,73,76,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,43,73,76,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,43,73,76,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [161]:
#Delete duplicated records and keep the first occurrences 
rt = rt.drop_duplicates(subset=['rotten_tomatoes_link'],keep='first')

In [162]:
rt.shape

(17706, 29)

In [163]:
# Check if there are any duplicate rows left
rt[rt.duplicated()== True].head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content


####  Drop the unnecessary columns

In [164]:
# dropp unnecessary column
# These columns are not required because these are descriptive information, which are not used for analysis.
rt = rt.drop(['movie_info', 'critics_consensus', 'publisher_name', 'critic_name', 'review_type', 'top_critic', 
              'review_content', 'review_score', 'review_date'],axis=1)

In [165]:
#revised dataset after revoming the unnecessary columns
rt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17706 entries, 0 to 1129879
Data columns (total 20 columns):
rotten_tomatoes_link                17706 non-null object
movie_title                         17706 non-null object
content_rating                      17706 non-null object
genres                              17687 non-null object
directors                           17512 non-null object
authors                             16165 non-null object
actors                              17355 non-null object
original_release_date               16542 non-null object
streaming_release_date              17323 non-null object
runtime                             17392 non-null float64
production_company                  17209 non-null object
tomatometer_status                  17662 non-null object
tomatometer_rating                  17662 non-null float64
tomatometer_count                   17662 non-null float64
audience_status                     17259 non-null object
audience_rating 

In [166]:
#find out the null percentage after dropping the columns
round(rt.isnull().sum().sort_values(ascending = False)/len(rt)*100,2)


authors                             8.70
original_release_date               6.57
production_company                  2.81
audience_status                     2.52
streaming_release_date              2.16
actors                              1.98
runtime                             1.77
audience_count                      1.67
audience_rating                     1.67
directors                           1.10
tomatometer_count                   0.25
tomatometer_rating                  0.25
tomatometer_status                  0.25
genres                              0.11
movie_title                         0.00
content_rating                      0.00
tomatometer_rotten_critics_count    0.00
tomatometer_fresh_critics_count     0.00
tomatometer_top_critics_count       0.00
rotten_tomatoes_link                0.00
dtype: float64

In [167]:
rt.shape

(17706, 20)

**Observation:**<br>
After removing the redundant columns, 17706 entries with 20 columns are available
<br>
But the columns `authors`,`critic_name`,`original_release_date` have more than 5% of null values

####  In this step, replace the remaining null values with 'unknown' in categorical columns

In [168]:
rt.fillna({'authors':'Unknown', 'audience_status':'Unknown', 'directors':'Unknown', 'actors': 'Unknown', 
             'production_company': 'Unknown', 'critic_name':'Unknown'}, inplace=True)

####  Extract the Year from the release_date

In [169]:
#extract the year from the release date
rt['streaming_year'] = pd.DatetimeIndex(rt['streaming_release_date']).year
rt['publish_year'] = pd.DatetimeIndex(rt['original_release_date']).year

####  Convert the columns type from float to int

In [170]:
#convert the the columns type from float to int

rt["streaming_year"] = rt["streaming_year"].astype('Int64')
rt["publish_year"] = rt["publish_year"].astype('Int64')
rt["runtime"] = rt["runtime"].astype('Int64')
rt["audience_count"] = rt["audience_count"].astype('Int64')
rt["tomatometer_count"] = rt["tomatometer_count"].astype('Int64')

#### Fix the year columns in a customised index for clear undersatnding

In [171]:
# defining the columns name to these columns in a customised position
publish_year = rt["publish_year"]

In [172]:
rt.insert(loc = 3, column= 'published_year', value= publish_year)

In [173]:
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,genres,directors,authors,actors,original_release_date,streaming_release_date,...,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,streaming_year,publish_year
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,...,49.0,149,Spilled,53.0,254421,43,73,76,2015,2010
149,m/0878835,Please Give,R,2010,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,2012-09-04,...,87.0,142,Upright,64.0,11574,44,123,19,2012,2010


In [174]:
#drop the year columns which are not required
rt = rt.drop(['original_release_date', 'streaming_release_date',
             'publish_year'],axis=1)

In [175]:
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,genres,directors,authors,actors,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,streaming_year
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",119,20th Century Fox,Rotten,49.0,149,Spilled,53.0,254421,43,73,76,2015
149,m/0878835,Please Give,R,2010,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",90,Sony Pictures Classics,Certified-Fresh,87.0,142,Upright,64.0,11574,44,123,19,2012


- Check the total number of values in terms of `movie_title` for the columns `genres` and `production_compoany`

In [176]:
#check the total number of different types of genres according to the movie title in descending manner
genres_group= rt.groupby("genres").movie_title.count().sort_values(ascending=False)
genres_group.head()

genres
Drama                               1887
Comedy                              1262
Comedy, Drama                        863
Drama, Mystery & Suspense            729
Art House & International, Drama     589
Name: movie_title, dtype: int64

> We can see that there are many different varieties of genre groups which are similar. We will split these comma separated genres for easier analytics and grouping

In [177]:
#check the production company produces the number of movies
rt.groupby('production_company').movie_title.count().sort_values(ascending = False).head()

production_company
Paramount Pictures       517
Warner Bros. Pictures    509
Unknown                  506
Universal Pictures       495
20th Century Fox         423
Name: movie_title, dtype: int64

> We can observe that the Production Company Paramount Pictures and Warner Bros have produced the highest number of movies as per the RT dataset

In [178]:
# Check if any of the columns have only one value throughout the dataset
rt.loc[:,rt.nunique()==1].columns

Index([], dtype='object')

####  Normalize the values of both of the rating columns

In [179]:
# normalize the rating columns
rating_list = ['tomatometer_rating', 'audience_rating']
rt[['tomatometer_rating_new', 'audience_rating_new']] = rt[rating_list]/10

In [180]:
# Drop original rating column because we have normalized ratings
rt = rt.drop(columns=rating_list)

In [181]:
# define the following columns to set in a customized index in the dataframe 
tomatometer_rating_new = rt["tomatometer_rating_new"]
audience_rating_new = rt["audience_rating_new"]
tomatometer_count = rt["tomatometer_count"]
audience_count = rt["audience_count"]

In [182]:
# insert the columns and renaming them in a cutomised location(index) for better analysis
rt.insert(loc =4, column= 'tomatometer_score', value= tomatometer_rating_new)
rt.insert(loc =5, column= 'audience_score', value= audience_rating_new)
rt.insert(loc =6, column= 'tomatometer_counts', value= tomatometer_count)
rt.insert(loc =7, column= 'audience_counts', value= audience_count)

In [183]:
# drop the columns which are not required
rating_list = ['tomatometer_rating_new', 'audience_rating_new']
rt = rt.drop(columns=rating_list)
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,runtime,production_company,tomatometer_status,tomatometer_count,audience_status,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,streaming_year
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010,4.9,5.3,149,254421,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,...,119,20th Century Fox,Rotten,149,Spilled,254421,43,73,76,2015
149,m/0878835,Please Give,R,2010,8.7,6.4,142,11574,Comedy,Nicole Holofcener,...,90,Sony Pictures Classics,Certified-Fresh,142,Upright,11574,44,123,19,2012


####  Calculate the weighted Average Vote

In [184]:
# count the total number of votes got from the critics and user
rt['total_votes_rt'] = rt['tomatometer_counts']+rt['audience_counts']

In [185]:
# generate a formula to calculate the 'weighted_avg_vote' using the columns 'tomatometer_score' and 'audience_score'
rt['weighted_average_vote']= np.where(rt['tomatometer_counts'].notnull() & rt['audience_counts'].notnull(), 
                                      (((rt['tomatometer_count']*rt['tomatometer_score'])+(rt['audience_count']*rt['audience_score']))/rt['total_votes_rt']),
                                      rt['total_votes_rt'])

In [186]:
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,tomatometer_status,tomatometer_count,audience_status,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,streaming_year,total_votes_rt,weighted_average_vote
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010,4.9,5.3,149,254421,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,...,Rotten,149,Spilled,254421,43,73,76,2015,254570,5.29977
149,m/0878835,Please Give,R,2010,8.7,6.4,142,11574,Comedy,Nicole Holofcener,...,Certified-Fresh,142,Upright,11574,44,123,19,2012,11716,6.42788


**-Try to get only 1 value after the decimal point for the values of `weighted_average_vote` column**

In [187]:
# give only 1 value after the decimal point
rt['weighted_average_vote'] = np.around(rt['weighted_average_vote'].astype(np.double),1)
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,tomatometer_status,tomatometer_count,audience_status,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,streaming_year,total_votes_rt,weighted_average_vote
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010,4.9,5.3,149,254421,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,...,Rotten,149,Spilled,254421,43,73,76,2015,254570,5.3
149,m/0878835,Please Give,R,2010,8.7,6.4,142,11574,Comedy,Nicole Holofcener,...,Certified-Fresh,142,Upright,11574,44,123,19,2012,11716,6.4


**-Get the unique values for the columns `runtime`, `streaming_year` and `published_year` and check for anomalies**


In [188]:
# find the unique records for runtime
rt.runtime.unique()

<IntegerArray>
[119,  90, 122,  95, 127, 109,  80,  92, 103,  97,
 ...
  52,  55,  59, 238, 206,  57,  15,  30,  41, 255]
Length: 191, dtype: Int64

In [189]:
# find the unique records for streaming_year
rt.streaming_year.unique()

<IntegerArray>
[2015, 2012, 2014, 2017, 2016, 2013, 2008, 2018,  NaN, 2005, 2006, 2020, 2007,
 2010, 2011, 2009, 2019, 2003, 2000, 2002, 2004, 1998, 2001, 1999]
Length: 24, dtype: Int64

In [190]:
# find the unique records for published_year
rt.published_year.unique()

<IntegerArray>
[2010, 1979, 1957, 1954, 2008, 1935, 2002, 1930, 1988, 2005,
 ...
 1998, 1995, 1996, 2001, 2016, 2019, 2018, 2020, 1915, 1914]
Length: 106, dtype: Int64

#### Observation
- All the values in runtime, streaming year and published year seem to be valid

####  Split the columns for Genres

In [191]:
# Strip the leading and lagging spaces in the dataset
rt =  rt.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [192]:
# Rename the names of the `genres` that are similar to each other a name that is common to both IMDB and RT 
# for easy analytics
rt["genres"] = rt["genres"].str.replace("Mystery & Suspense","Mystery")                                                                          
rt["genres"] = rt["genres"].str.replace("Sports & Fitness","Sport")                                        
rt["genres"] = rt["genres"].str.replace("Musical & Performing Arts","Musical")     
rt["genres"] = rt["genres"].str.replace("Science Fiction","Sci-Fi")
rt["genres"] = rt["genres"].str.replace("Art House & International","ArtHouse and International")
rt["genres"] = rt["genres"].str.replace("Faith & Spirituality"," Faith and Spirituality")
rt["genres"] = rt["genres"].str.replace("Gay & Lesbian"," Gay and Lesbian")
rt["genres"] = rt["genres"].str.replace(" Anime & Manga"," Anime and Manga")


In [193]:
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,tomatometer_status,tomatometer_count,audience_status,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,streaming_year,total_votes_rt,weighted_average_vote
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010.0,4.9,5.3,149.0,254421.0,"Action & Adventure, Comedy, Drama, Sci-Fi & Fa...",Chris Columbus,...,Rotten,149.0,Spilled,254421.0,43,73,76,2015.0,254570.0,5.3
149,m/0878835,Please Give,R,2010.0,8.7,6.4,142.0,11574.0,Comedy,Nicole Holofcener,...,Certified-Fresh,142.0,Upright,11574.0,44,123,19,2012.0,11716.0,6.4


In [194]:
# Split the genres column 
rt = rt.join(rt['genres'].str.split(r"&|,", expand=True).add_prefix('genre'))

In [195]:
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,genre0,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010.0,4.9,5.3,149.0,254421.0,"Action & Adventure, Comedy, Drama, Sci-Fi & Fa...",Chris Columbus,...,Action,Adventure,Comedy,Drama,Sci-Fi,Fantasy,None,None,None,None
149,m/0878835,Please Give,R,2010.0,8.7,6.4,142.0,11574.0,Comedy,Nicole Holofcener,...,Comedy,None,None,None,None,None,None,None,None,None


In [196]:
round(rt.isnull().sum().sort_values(ascending = False)/len(rt)*100,2)

genre9                              99.99
genre8                              99.95
genre7                              99.67
genre6                              98.75
genre5                              96.07
genre4                              89.68
genre3                              76.77
genre2                              54.80
genre1                              25.05
published_year                       6.57
streaming_year                       2.16
runtime                              1.77
total_votes_rt                       1.72
weighted_average_vote                1.72
audience_count                       1.67
audience_counts                      1.67
audience_score                       1.67
tomatometer_count                    0.25
tomatometer_status                   0.25
tomatometer_score                    0.25
tomatometer_counts                   0.25
genres                               0.11
genre0                               0.11
movie_title                       

- The columns `genre9`, `genre8`, `genre7`,`genre6`, `genre5`, `genre4` have very large percentage null which will have no use for data analysis. Therefore, drop these columns

In [197]:
rt = rt.drop(['genre9', 'genre8', 'genre7', 'genre6', 'genre5', 'genre4'],axis=1)


#### Extract the director´s name from `directors` column because there are more than one directors for particular movie

In [198]:
# Now extract first 3 directors
rt['director0'] = rt['directors'].str.split(',').str[0]
rt['director1'] = rt['directors'].str.split(',').str[1]
rt['director2'] = rt['directors'].str.split(',').str[2]

In [199]:
rt.head()

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,streaming_year,total_votes_rt,weighted_average_vote,genre0,genre1,genre2,genre3,director0,director1,director2
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010.0,4.9,5.3,149.0,254421.0,"Action & Adventure, Comedy, Drama, Sci-Fi & Fa...",Chris Columbus,...,2015.0,254570.0,5.3,Action,Adventure,Comedy,Drama,Chris Columbus,NaN,NaN
149,m/0878835,Please Give,R,2010.0,8.7,6.4,142.0,11574.0,Comedy,Nicole Holofcener,...,2012.0,11716.0,6.4,Comedy,None,None,None,Nicole Holofcener,NaN,NaN
291,m/10,10,R,1979.0,6.7,5.3,24.0,14684.0,"Comedy, Romance",Blake Edwards,...,2014.0,14708.0,5.3,Comedy,Romance,None,None,Blake Edwards,NaN,NaN
315,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),NR,1957.0,10.0,9.7,54.0,105386.0,"Classics, Drama",Sidney Lumet,...,2017.0,105440.0,9.7,Classics,Drama,None,None,Sidney Lumet,NaN,NaN
369,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",G,1954.0,8.9,7.4,27.0,68918.0,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,...,2016.0,68945.0,7.4,Action,Adventure,Drama,Kids,Richard Fleischer,NaN,NaN


####  Extract the author name from `authors` column  because there are more than one authors for particular movie

In [200]:
# Now extract first 3 authors
rt['author0'] = rt['authors'].str.split(',').str[0]
rt['author1'] = rt['authors'].str.split(',').str[1]
rt['author2'] = rt['authors'].str.split(',').str[2]

In [201]:
rt.head()

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,genre0,genre1,genre2,genre3,director0,director1,director2,author0,author1,author2
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010.0,4.9,5.3,149.0,254421.0,"Action & Adventure, Comedy, Drama, Sci-Fi & Fa...",Chris Columbus,...,Action,Adventure,Comedy,Drama,Chris Columbus,NaN,NaN,Craig Titley,Chris Columbus,Rick Riordan
149,m/0878835,Please Give,R,2010.0,8.7,6.4,142.0,11574.0,Comedy,Nicole Holofcener,...,Comedy,None,None,None,Nicole Holofcener,NaN,NaN,Nicole Holofcener,NaN,NaN
291,m/10,10,R,1979.0,6.7,5.3,24.0,14684.0,"Comedy, Romance",Blake Edwards,...,Comedy,Romance,None,None,Blake Edwards,NaN,NaN,Blake Edwards,NaN,NaN
315,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),NR,1957.0,10.0,9.7,54.0,105386.0,"Classics, Drama",Sidney Lumet,...,Classics,Drama,None,None,Sidney Lumet,NaN,NaN,Reginald Rose,NaN,NaN
369,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",G,1954.0,8.9,7.4,27.0,68918.0,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,...,Action,Adventure,Drama,Kids,Richard Fleischer,NaN,NaN,Earl Felton,NaN,NaN


####  Extract the actors name from `actors` column  because there are more than one actors for particular movie


In [202]:
# Now extract first 3 actors
rt['actors0'] = rt['actors'].str.split(',').str[0]
rt['actors1'] = rt['actors'].str.split(',').str[1]
rt['actors2'] = rt['actors'].str.split(',').str[1]

In [203]:
rt.head()

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,genre3,director0,director1,director2,author0,author1,author2,actors0,actors1,actors2
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,2010.0,4.9,5.3,149.0,254421.0,"Action & Adventure, Comedy, Drama, Sci-Fi & Fa...",Chris Columbus,...,Drama,Chris Columbus,NaN,NaN,Craig Titley,Chris Columbus,Rick Riordan,Logan Lerman,Brandon T. Jackson,Brandon T. Jackson
149,m/0878835,Please Give,R,2010.0,8.7,6.4,142.0,11574.0,Comedy,Nicole Holofcener,...,None,Nicole Holofcener,NaN,NaN,Nicole Holofcener,NaN,NaN,Catherine Keener,Amanda Peet,Amanda Peet
291,m/10,10,R,1979.0,6.7,5.3,24.0,14684.0,"Comedy, Romance",Blake Edwards,...,None,Blake Edwards,NaN,NaN,Blake Edwards,NaN,NaN,Dudley Moore,Bo Derek,Bo Derek
315,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),NR,1957.0,10.0,9.7,54.0,105386.0,"Classics, Drama",Sidney Lumet,...,None,Sidney Lumet,NaN,NaN,Reginald Rose,NaN,NaN,Martin Balsam,John Fiedler,John Fiedler
369,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",G,1954.0,8.9,7.4,27.0,68918.0,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,...,Kids,Richard Fleischer,NaN,NaN,Earl Felton,NaN,NaN,James Mason,Kirk Douglas,Kirk Douglas


In [204]:
round(rt.isnull().sum().sort_values(ascending = False)/len(rt)*100,2)

director2                           98.45
director1                           89.63
author2                             82.81
genre3                              76.77
author1                             54.82
genre2                              54.80
genre1                              25.05
published_year                       6.57
actors2                              3.67
actors1                              3.67
streaming_year                       2.16
runtime                              1.77
total_votes_rt                       1.72
weighted_average_vote                1.72
audience_count                       1.67
audience_counts                      1.67
audience_score                       1.67
tomatometer_score                    0.25
tomatometer_counts                   0.25
tomatometer_count                    0.25
tomatometer_status                   0.25
genre0                               0.11
genres                               0.11
directors                         

-Drop the redundant columns and also the column `director2` because it has 98% null values 

In [205]:
# drop the redundant columns
rt = rt.drop(['director2', 'audience_count', 'tomatometer_count'],axis=1)

In [206]:
round(rt.isnull().sum().sort_values(ascending = False)/len(rt)*100,2)

director1                           89.63
author2                             82.81
genre3                              76.77
author1                             54.82
genre2                              54.80
genre1                              25.05
published_year                       6.57
actors2                              3.67
actors1                              3.67
streaming_year                       2.16
runtime                              1.77
weighted_average_vote                1.72
total_votes_rt                       1.72
audience_counts                      1.67
audience_score                       1.67
tomatometer_counts                   0.25
tomatometer_status                   0.25
tomatometer_score                    0.25
genre0                               0.11
genres                               0.11
directors                            0.00
content_rating                       0.00
movie_title                          0.00
tomatometer_top_critics_count     

In [207]:
# Strip again in whole the dataset to remove leading and lagging spaces
rt =  rt.applymap(lambda x: x.strip() if isinstance(x, str) else x)

### Filter out the movies   where the number of votes from audience is greatger than 100 

In [208]:
rt = rt[rt.audience_counts > 100].sort_values(by=['audience_counts'], ascending=False)

In [209]:
rt.head(3)

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,genre2,genre3,director0,director1,author0,author1,author2,actors0,actors1,actors2
1043383,m/titanic,Titanic,PG-13,1997.0,8.9,6.9,192.0,35797635.0,"Drama, Romance",James Cameron,...,None,None,James Cameron,NaN,James Cameron,NaN,NaN,Leonardo DiCaprio,Kate Winslet,Kate Winslet
989681,m/the_lord_of_the_rings_the_return_of_the_king,The Lord of the Rings: The Return of the King,PG-13,2003.0,9.3,8.6,275.0,34679773.0,"Action & Adventure, Classics, Sci-Fi & Fantasy",Peter Jackson,...,Classics,Sci-Fi,Peter Jackson,NaN,Fran Walsh,Philippa Boyens,Peter Jackson,Elijah Wood,Sean Astin,Sean Astin
853830,m/spiderman,Spider-Man,PG-13,2002.0,9.0,6.7,245.0,34297354.0,"Action & Adventure, Mystery, Sci-Fi & Fantasy",Sam Raimi,...,Mystery,Sci-Fi,Sam Raimi,NaN,David Koepp,NaN,NaN,Tobey Maguire,Kirsten Dunst,Kirsten Dunst


In [210]:
# reset the dataframe index
rt = rt.reset_index()

In [211]:
rt = rt.drop(['index'],axis=1)
rt.head(2)

,rotten_tomatoes_link,movie_title,content_rating,published_year,tomatometer_score,audience_score,tomatometer_counts,audience_counts,genres,directors,...,genre2,genre3,director0,director1,author0,author1,author2,actors0,actors1,actors2
0,m/titanic,Titanic,PG-13,1997.0,8.9,6.9,192.0,35797635.0,"Drama, Romance",James Cameron,...,None,None,James Cameron,NaN,James Cameron,NaN,NaN,Leonardo DiCaprio,Kate Winslet,Kate Winslet
1,m/the_lord_of_the_rings_the_return_of_the_king,The Lord of the Rings: The Return of the King,PG-13,2003.0,9.3,8.6,275.0,34679773.0,"Action & Adventure, Classics, Sci-Fi & Fantasy",Peter Jackson,...,Classics,Sci-Fi,Peter Jackson,NaN,Fran Walsh,Philippa Boyens,Peter Jackson,Elijah Wood,Sean Astin,Sean Astin


In [212]:
rt.shape

(16288, 34)

> Now we have clean RT dataset that can be merged with IMDB

## 2.5 Merge IMDB and Rotten Tomatoes to get movies dataset

In [213]:
# Full outer join to get all the information from both the datasets
movies = pd.merge(imdb, rt, how = 'outer', left_on = 'title', right_on = 'movie_title')

In [214]:
# Checking the shape of the dataset
movies.shape

(96874, 95)

In [215]:
# Naturally, there are a couple of columns that are redundant, which need to be removed
# Merge columns that contain same information but present in different columns (imdb and rt) into a single
movies = movies.assign(title = movies.title.fillna(movies.movie_title),
                      year = movies.year.fillna(movies.published_year),
                      duration = movies.duration.fillna(movies.runtime),
                      genre0 = movies.genre0_x.fillna(movies.genre0_y),
                      genre1 = movies.genre1_x.fillna(movies.genre1_y),
                      genre2 = movies.genre2_x.fillna(movies.genre2_y),
                      director0 = movies.director0_x.fillna(movies.director0_y),
                      director1 = movies.director1_x.fillna(movies.director1_y),
                      writer0 = movies.writer0.fillna(movies.author0),
                      writer1 = movies.writer1.fillna(movies.author1),
                      actors0 = movies.actors0_x.fillna(movies.actors0_y),
                      actors1 = movies.actors1_x.fillna(movies.actors1_y),
                      actors2 = movies.actors2_x.fillna(movies.actors2_y),
                      production_company = movies.production_company_x.fillna(movies.production_company_y))

In [216]:
# Finally drop all the unnecessary columns from the cleaned movies dataset
movies.drop(['movie_title', 'genre', 'genres', 'runtime', 'director', 'directors', 
             'writer', 'authors', 'production_company_x', 'production_company_y', 'actors_x', 'actors_y', 'genre0_x', 
             'genre1_x', 'genre2_x', 'genre0_y', 'genre1_y', 'genre2_y', 'director0_x', 'director1_x', 'director0_y', 
             'director1_y', 'author0', 'author1', 'author2', 'actors0_x', 'actors1_x', 'actors2_x', 'actors0_y', 
             'actors1_y', 'actors2_y', 'rotten_tomatoes_link'], axis=1, inplace=True)

In [217]:
# Create a new average vote column to calculate weighted average from IMDB and RT
# Before that, we null in weighted_average_vote_x with value in weighted_average_vote_y and vice verca, 
# so that when the average is computed, we get correct result
movies['weighted_average_vote_x'].fillna(movies['weighted_average_vote_y'], inplace = True)
movies['weighted_average_vote_y'].fillna(movies['weighted_average_vote_x'], inplace = True)

In [218]:
# Create new ratings column
movies['ratings'] = (movies['weighted_average_vote_x'] + movies['weighted_average_vote_y'])/2

In [219]:
# Drop the weighted average of imdb and rt
movies.drop(['weighted_average_vote_x', 'weighted_average_vote_y'], axis=1, inplace=True)

In [220]:
# Similarly, add the total number of votes from both imdb and rt to get the complete picture.
# But before adding the columns, fill nulls with 0 to avoid computational errors
movies['total_votes_imdb'].fillna(0, inplace = True)
movies['total_votes_rt'].fillna(0, inplace = True)

In [221]:
# Create new votes column
movies['votes'] = movies['total_votes_imdb'] + movies['total_votes_rt']

In [222]:
# Drop the total_votes column that belongs to rt dataset
movies.drop(['total_votes_imdb', 'total_votes_rt'], axis=1, inplace=True)

In [223]:
# Check out the dataset
movies.head(2)

,title,year,duration,country,language,reviews_from_users,reviews_from_critics,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,...,genre1,genre2,director0,director1,actors0,actors1,actors2,production_company,ratings,votes
0,Miss Jerry,1894.0,45.0,USA,Unknown,1.0,2.0,6.0,38.0,5.7,...,NaN,NaN,Alexander Black,NaN,Blanche Bayliss,William Courtenay,Chauncey Depew,Alexander Black Photoplays,5.9,154.0
1,The Story of the Kelly Gang,1906.0,70.0,Australia,Unknown,7.0,7.0,6.1,114.0,6.0,...,Crime,Drama,Charles Tait,NaN,Elizabeth Tait,John Tait,Norman Campbell,J. and N. Tait,6.1,589.0


In [224]:
# Check the shape of the dataset
movies.shape

(96874, 70)

> The cleaned movies dataset is now ready to be pushed into the DB

# 3. Push Data To DB

In [225]:
# Storing the connection details in variables
username = os.environ.get("mysql_username")
password = os.environ.get("mysql_password")
hostname = os.environ.get("mysql_hostname")
database = os.environ.get("mysql_database")

# Creating connection string
connection_string = 'mysql+pymysql://' + username + ':' + password + '@' + hostname + '/' + database

In [227]:
# Create SQL engine to connect to the DB named netflix_analysis
sqlEngine = create_engine(connection_string, pool_recycle=3600)
# Staring the connection session
dbConnection = sqlEngine.connect()
# Push all datasets into their respective tables. The code will fail if the table already exists
netflix_titles.to_sql("netflix_titles", dbConnection, if_exists='fail')
prime_IMDb_final.to_sql("prime_IMDb_final", dbConnection, if_exists='fail')
movies.to_sql("movies", dbConnection, if_exists='fail');
# Close the connection
dbConnection.close()